# PRÀCTICA BLOC II (INTEL·LIGÈNCIA ARTIFICIAL) 



Miquel Vives Marcus<br />
Repositori Git: https://github.com/miqvm/practica-ML-21722


## Introducció

FIFA és un videojoc que actua com un simulador de futbol. Els jugadors que hi apareixen estan basats en jugadors reals, els quals tenen uns atributs definits segons les seves habilitats futbolístiques en la vida real. Un d'aquests atributs és el preu de mercat, important per realitzar transaccions de compra-venda dins del joc. En aquesta pràctica, a partir d'un dataset amb tots els jugadors que es troben en el joc, es farà un model de predicció del preu de mercat amb un algorisme de Machine Learning. 

## Importam les llibreries

Per resoldre aquest problema utilitzarem la llibraria ``sklearn``, per realitzar tot el relacionat amb el machine learning; ``pandas`` per llegir i operar sobre el dataset, i ``numpy`` per operar amb arrays i matrius. 

S'importen les llibreries esmentades:

In [1]:
import os

from sklearn.model_selection import train_test_split
from sklearn import linear_model
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

import pandas as pd
import numpy as np

## Llegim les dades

Llegim i mostram el dataset importat usant ``pd.read_csv()`` i una utilitat de la llibraria ``os`` per extreure la direcció d'on es troba el dataset.

In [2]:
df = pd.read_csv(os.path.join("..", "in", "fifa.csv"))
df.head()

,Unnamed: 0,ID,Name,Age,Photo,Nationality,Flag,Overall,Potential,Club,...,Composure,Marking,StandingTackle,SlidingTackle,GKDiving,GKHandling,GKKicking,GKPositioning,GKReflexes,Release Clause
0,0,158023,L. Messi,31,https://cdn.sofifa.org/players/4/19/158023.png,Argentina,https://cdn.sofifa.org/flags/52.png,94,94,FC Barcelona,...,96.0,33.0,28.0,26.0,6.0,11.0,15.0,14.0,8.0,€226.5M
1,1,20801,Cristiano Ronaldo,33,https://cdn.sofifa.org/players/4/19/20801.png,Portugal,https://cdn.sofifa.org/flags/38.png,94,94,Juventus,...,95.0,28.0,31.0,23.0,7.0,11.0,15.0,14.0,11.0,€127.1M
2,2,190871,Neymar Jr,26,https://cdn.sofifa.org/players/4/19/190871.png,Brazil,https://cdn.sofifa.org/flags/54.png,92,93,Paris Saint-Germain,...,94.0,27.0,24.0,33.0,9.0,9.0,15.0,15.0,11.0,€228.1M
3,3,193080,De Gea,27,https://cdn.sofifa.org/players/4/19/193080.png,Spain,https://cdn.sofifa.org/flags/45.png,91,93,Manchester United,...,68.0,15.0,21.0,13.0,90.0,85.0,87.0,88.0,94.0,€138.6M
4,4,192985,K. De Bruyne,27,https://cdn.sofifa.org/players/4/19/192985.png,Belgium,https://cdn.sofifa.org/flags/7.png,91,92,Manchester City,...,88.0,68.0,58.0,51.0,15.0,13.0,5.0,10.0,13.0,€196.4M



## Analisis de les dades

En aquest bloc, s'identificaran els diferents atributs que tenen els registres del dataset, s'eliminaran els irrelevants, i es convertiran a un format apte per treballar els que no ho estiguin.

Primerament, mostrarem el dataset per complet:

In [3]:
pd.set_option('display.max_columns', None)
df.head()

,Unnamed: 0,ID,Name,Age,Photo,Nationality,Flag,Overall,Potential,Club,Club Logo,Value,Wage,Special,Preferred Foot,International Reputation,Weak Foot,Skill Moves,Work Rate,Body Type,Real Face,Position,Jersey Number,Joined,Loaned From,Contract Valid Until,Height,Weight,LS,ST,RS,LW,LF,CF,RF,RW,LAM,CAM,RAM,LM,LCM,CM,RCM,RM,LWB,LDM,CDM,RDM,RWB,LB,LCB,CB,RCB,RB,Crossing,Finishing,HeadingAccuracy,ShortPassing,Volleys,Dribbling,Curve,FKAccuracy,LongPassing,BallControl,Acceleration,SprintSpeed,Agility,Reactions,Balance,ShotPower,Jumping,Stamina,Strength,LongShots,Aggression,Interceptions,Positioning,Vision,Penalties,Composure,Marking,StandingTackle,SlidingTackle,GKDiving,GKHandling,GKKicking,GKPositioning,GKReflexes,Release Clause
0,0,158023,L. Messi,31,https://cdn.sofifa.org/players/4/19/158023.png,Argentina,https://cdn.sofifa.org/flags/52.png,94,94,FC Barcelona,https://cdn.sofifa.org/teams/2/light/241.png,€110.5M,€565K,2202,Left,5.0,4.0,4.0,Medium/ Medium,Messi,Yes,RF,10.0,"Jul 1, 2004",NaN,2021,5'7,159lbs,88+2,88+2,88+2,92+2,93+2,93+2,93+2,92+2,93+2,93+2,93+2,91+2,84+2,84+2,84+2,91+2,64+2,61+2,61+2,61+2,64+2,59+2,47+2,47+2,47+2,59+2,84.0,95.0,70.0,90.0,86.0,97.0,93.0,94.0,87.0,96.0,91.0,86.0,91.0,95.0,95.0,85.0,68.0,72.0,59.0,94.0,48.0,22.0,94.0,94.0,75.0,96.0,33.0,28.0,26.0,6.0,11.0,15.0,14.0,8.0,€226.5M
1,1,20801,Cristiano Ronaldo,33,https://cdn.sofifa.org/players/4/19/20801.png,Portugal,https://cdn.sofifa.org/flags/38.png,94,94,Juventus,https://cdn.sofifa.org/teams/2/light/45.png,€77M,€405K,2228,Right,5.0,4.0,5.0,High/ Low,C. Ronaldo,Yes,ST,7.0,"Jul 10, 2018",NaN,2022,6'2,183lbs,91+3,91+3,91+3,89+3,90+3,90+3,90+3,89+3,88+3,88+3,88+3,88+3,81+3,81+3,81+3,88+3,65+3,61+3,61+3,61+3,65+3,61+3,53+3,53+3,53+3,61+3,84.0,94.0,89.0,81.0,87.0,88.0,81.0,76.0,77.0,94.0,89.0,91.0,87.0,96.0,70.0,95.0,95.0,88.0,79.0,93.0,63.0,29.0,95.0,82.0,85.0,95.0,28.0,31.0,23.0,7.0,11.0,15.0,14.0,11.0,€127.1M
2,2,190871,Neymar Jr,26,https://cdn.sofifa.org/players/4/19/190871.png,Brazil,https://cdn.sofifa.org/flags/54.png,92,93,Paris Saint-Germain,https://cdn.sofifa.org/teams/2/light/73.png,€118.5M,€290K,2143,Right,5.0,5.0,5.0,High/ Medium,Neymar,Yes,LW,10.0,"Aug 3, 2017",NaN,2022,5'9,150lbs,84+3,84+3,84+3,89+3,89+3,89+3,89+3,89+3,89+3,89+3,89+3,88+3,81+3,81+3,81+3,88+3,65+3,60+3,60+3,60+3,65+3,60+3,47+3,47+3,47+3,60+3,79.0,87.0,62.0,84.0,84.0,96.0,88.0,87.0,78.0,95.0,94.0,90.0,96.0,94.0,84.0,80.0,61.0,81.0,49.0,82.0,56.0,36.0,89.0,87.0,81.0,94.0,27.0,24.0,33.0,9.0,9.0,15.0,15.0,11.0,€228.1M
3,3,193080,De Gea,27,https://cdn.sofifa.org/players/4/19/193080.png,Spain,https://cdn.sofifa.org/flags/45.png,91,93,Manchester United,https://cdn.sofifa.org/teams/2/light/11.png,€72M,€260K,1471,Right,4.0,3.0,1.0,Medium/ Medium,Lean,Yes,GK,1.0,"Jul 1, 2011",NaN,2020,6'4,168lbs,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,17.0,13.0,21.0,50.0,13.0,18.0,21.0,19.0,51.0,42.0,57.0,58.0,60.0,90.0,43.0,31.0,67.0,43.0,64.0,12.0,38.0,30.0,12.0,68.0,40.0,68.0,15.0,21.0,13.0,90.0,85.0,87.0,88.0,94.0,€138.6M
4,4,192985,K. De Bruyne,27,https://cdn.sofifa.org/players/4/19/192985.png,Belgium,https://cdn.sofifa.org/flags/7.png,91,92,Manchester City,https://cdn.sofifa.org/teams/2/light/10.png,€102M,€355K,2281,Right,4.0,5.0,4.0,High/ High,Normal,Yes,RCM,7.0,"Aug 30, 2015",NaN,2023,5'11,154lbs,82+3,82+3,82+3,87+3,87+3,87+3,87+3,87+3,88+3,88+3,88+3,88+3,87+3,87+3,87+3,88+3,77+3,77+3,77+3,77+3,77+3,73+3,66+3,66+3,66+3,73+3,93.0,82.0,55.0,92.0,82.0,86.0,85.0,83.0,91.0,91.0,78.0,76.0,79.0,91.0,77.0,91.0,63.0,90.0,75.0,91.0,76.0,61.0,87.0,94.0,79.0,88.0,68.0,58.0,51.0,15.0,13.0,5.0,10.0,13.0,€196.4M


**Descripció dels atributs:**

``Unnamed 0:`` Posició del jugador en el dataset.<br />
``ID:`` Nombre identificador del jugador en la base de dades de FIFA.<br />
``Name:`` Nom complet del jugador.<br />
``Age:`` Edat del jugador.<br />
``Photo:`` Indica l'URL de la foto del jugador.<br />
``Nationality:`` País de naixament.<br />
``Flag:`` URL de la bandera del país de naixement.<br />
``Overall`` Mitja dels seus atributs, a major és el nombre, millor és el jugador.<br />
``Potential:`` Indica com de ràpid els atributs del jugador augmentaran i quan s'aturarà aquest creixement.<br />
``Club:`` Equip al que pertany.<br />
``Club Logo:`` URL de l'emblema del club.<br />
``Value:`` Preu de mercat, atribut a fer la predicció.<br />
``Wage:`` Salari del jugador.<br />
``Special:`` Indica si el jugador té una carta especial<br />
``Preferred Foot:`` Indica si es dretà o esquerrà.<br />
``International Reputation:`` De 0-5 indica quan de valorat és aquest jugador al reu del món.<br />
``Weak Foot:`` De 0-5 indica quant és la seva destresa amb el peu dolent.<br />
``Skill Moves:`` De 0-5 indica l'habilitat que té per fer filigranes.<br />
``Work Rate:`` Mesurat amb *Low/Medium/High* indica el ritme de treball.<br />
``Body Type:`` Model del cos 3D en el FIFA.<br />
``Real Face:`` Indica si la cara que apareix en el joc es la seva real.<br />
``Position:`` Indica la seva posició en el camp de joc.<br />
``Jersey Number:`` Nombre que duu a la samarreta en el seu equip.<br />
``Joined:`` Quan de temps fa que aquest jugador està en el club que s'indica.<br />
``Loaned From:`` Indica si el jugador ha estat cedit per un altre club.<br />
``Contract Valid Until:`` Data d'expiració del contracte.<br />
``Height:`` Altura del jugador.<br />
``Weight:`` Pes del jugador.<br />
``LS:`` Indica de 0-100 l'habilitat del jugador en la posició de "Left striker".<br />
``ST:`` Indica de 0-100 l'habilitat del jugador en la posició de "Striker".<br />
``RS:`` Indica de 0-100 l'habilitat del jugador en la posició de "Right striker".<br />
``LW:`` Indica de 0-100 l'habilitat del jugador en la posició de "Left winger".<br />
``LF:`` Indica de 0-100 l'habilitat del jugador en la posició de "Left forward".<br />
``CF:`` Indica de 0-100 l'habilitat del jugador en la posició de "Central forward".<br />
``RF:`` Indica de 0-100 l'habilitat del jugador en la posició de "Right forward".<br />
``RW:`` Indica de 0-100 l'habilitat del jugador en la posició de "Right winger".<br />
``LAM:`` Indica de 0-100 l'habilitat del jugador en la posició de "Left attacking midfielder".<br />
``CAM:`` Indica de 0-100 l'habilitat del jugador en la posició de "Central attacking midfielder".<br />
``RAM:`` Indica de 0-100 l'habilitat del jugador en la posició de "Right attacking midfielder".<br />
``LM:`` Indica de 0-100 l'habilitat del jugador en la posició de "Left midfielder".<br />
``LCM:`` Indica de 0-100 l'habilitat del jugador en la posició de "Left central midfielder".<br />
``CM:`` Indica de 0-100 l'habilitat del jugador en la posició de "Central midfielder".<br />
``RCM:`` Indica de 0-100 l'habilitat del jugador en la posició de "Right central midfielder".<br />
``RM:`` Indica de 0-100 l'habilitat del jugador en la posició de "Right midfielder".<br />
``LWB:`` Indica de 0-100 l'habilitat del jugador en la posició de "Left winger back".<br />
``LDM:`` Indica de 0-100 l'habilitat del jugador en la posició de "Left defense midfielder".<br />
``CDM:`` Indica de 0-100 l'habilitat del jugador en la posició de "Central defense midfielder".<br />
``RDM:`` Indica de 0-100 l'habilitat del jugador en la posició de "Right defense midfielder".<br />
``RWB:`` Indica de 0-100 l'habilitat del jugador en la posició de "Right winger back".<br />
``LB:`` Indica de 0-100 l'habilitat del jugador en la posició de "Left back".<br />
``LCB:`` Indica de 0-100 l'habilitat del jugador en la posició de "Left central back".<br />
``CB:`` Indica de 0-100 l'habilitat del jugador en la posició de "Central back".<br />
``RCB:`` Indica de 0-100 l'habilitat del jugador en la posició de "Right central back".<br />
``RB:`` Indica de 0-100 l'habilitat del jugador en la posició de "Right back".<br />
``Crossing:``  Indica de 0-100 la presició del jugador en fer passades creuades.<br />
``Finishing:`` Indica de 0-100 la presició del jugador en fer xuts dins l'àrea.<br />
``HeadingAccuracy:`` Indica de 0-100 l'habilitat del jugador en fer remats de cap.<br />
``ShortPassing:`` Indica de 0-100 l'habilitat del jugador en fer passades a curta distància.<br />
``Volleys Dribbling:`` Indica de 0-100 l'habilitat del jugador en ragetetjar usant salvaments.<br />
``Curve:`` Indica de 0-100 l'habilitat del jugador en fer un tir amb curvatura.<br />
``FKAccuracy:``  Indica de 0-100 l'habilitat del jugador per xutar un tir lliure.<br />
``LongPassing:`` Indica de 0-100 l'habilitat del jugador en fer passades a llarga distància.<br />
``BallControl:`` Indica de 0-100 l'habilitat del jugador en controlar la pilota.<br />
``Acceleration:`` Indica de 0-100 l'habilitat l'acceleració del jugador corrent .<br />
``SprintSpeed:`` Indica de 0-100 la velocitat del jugador.<br />
``Agility Reactions:`` Indica de 0-100 l'agiliat del jugador.<br />
``Balance:`` Indica de 0-100 l'habilitat del jugador en mantenir l'equilibri després d'un cop.<br />
``ShotPower:`` Indica de 0-100 la potència de xut del jugador.<br />
``Jumping Stamina Strength:`` Indica de 0-100 l'altura que pot botar el jugador.<br />
``Long Shots:`` Indica de 0-100 la presició del jugador en xuts a llarga distància.<br />
``Aggression:`` Indica de 0-100 l'habilitat del jugador en realitzar una falta.<br />
``Interceptions:`` Indica de 0-100 l'habilitat del jugador en rebre i mantenir la pilota.<br />
``Positioning:`` Indica de 0-100 l'habilitat del jugador en posicionar-se en el camp de joc.<br />
``Vision:`` Indica de 0-100 l'habilitat del jugador en percebre als seus companys i l'entorn de joc durant el partit.<br />
``Penalties:`` Indica de 0-100 l'habilitat del jugador en xutar penaltis.<br />
``Composure:`` Indica de 0-100 l'habilitat del jugador en aguantar la presió frent als rivals.<br />
``Marking Standing Tackle:`` Indica de 0-100 l'habilitat del jugador en fer presió a un jugador rival.<br />
``SlidingTackle:`` Indica de 0-100 l'habilitat del jugador en realitzar una falta a un jugador rival.<br />
``GKDiving:`` De 0-100 habilitat del porter per agafar la pilota mentres ell està a l'aire.<br />
``GKHandling:`` De 0-100 habilitat del porter per aturar la pilota i mantenir-la en les seves mans.<br />
``GKKicking:`` De 0-100 habilitat del porter per xutar la pilota.<br />
``GKPositioning:`` De 0-100 habilitat del porter per recolocar-se una vegada aturi el primer xut a porta.<br />
``GKReflexes:`` De 0-100 reflexes del porter.<br />
``Release Clause:`` Quantitat monetaria per realitzar una clausula de recessió.<br />


Fent ``df.describe()`` veurem la suma total, la mitjana, la desviació típica, el valor mínim i màxim, i diferents percentils de les dades numèriques per veure quines són convenients mantenir i quines es poden eliminar.

In [4]:
df.describe()

,Unnamed: 0,ID,Age,Overall,Potential,Special,International Reputation,Weak Foot,Skill Moves,Jersey Number,Crossing,Finishing,HeadingAccuracy,ShortPassing,Volleys,Dribbling,Curve,FKAccuracy,LongPassing,BallControl,Acceleration,SprintSpeed,Agility,Reactions,Balance,ShotPower,Jumping,Stamina,Strength,LongShots,Aggression,Interceptions,Positioning,Vision,Penalties,Composure,Marking,StandingTackle,SlidingTackle,GKDiving,GKHandling,GKKicking,GKPositioning,GKReflexes
count,18207.000000,18207.000000,18207.000000,18207.000000,18207.000000,18207.000000,18159.000000,18159.000000,18159.000000,18147.000000,18159.000000,18159.000000,18159.000000,18159.000000,18159.000000,18159.000000,18159.000000,18159.000000,18159.000000,18159.000000,18159.000000,18159.000000,18159.000000,18159.000000,18159.000000,18159.000000,18159.000000,18159.000000,18159.000000,18159.000000,18159.000000,18159.000000,18159.000000,18159.000000,18159.000000,18159.000000,18159.000000,18159.000000,18159.000000,18159.000000,18159.000000,18159.000000,18159.000000,18159.000000
mean,9103.000000,214298.338606,25.122206,66.238699,71.307299,1597.809908,1.113222,2.947299,2.361308,19.546096,49.734181,45.550911,52.298144,58.686712,42.909026,55.371001,47.170824,42.863153,52.711933,58.369459,64.614076,64.726967,63.503607,61.836610,63.966573,55.460047,65.089432,63.219946,65.311967,47.109973,55.868991,46.698276,49.958478,53.400903,48.548598,58.648274,47.281623,47.697836,45.661435,16.616223,16.391596,16.232061,16.388898,16.710887
std,5256.052511,29965.244204,4.669943,6.908930,6.136496,272.586016,0.394031,0.660456,0.756164,15.947765,18.364524,19.525820,17.379909,14.699495,17.694408,18.910371,18.395264,17.478763,15.327870,16.686595,14.927780,14.649953,14.766049,9.010464,14.136166,17.237958,11.820044,15.894741,12.557000,19.260524,17.367967,20.696909,19.529036,14.146881,15.704053,11.436133,19.904397,21.664004,21.289135,17.695349,16.906900,16.502864,17.034669,17.955119
min,0.000000,16.000000,16.000000,46.000000,48.000000,731.000000,1.000000,1.000000,1.000000,1.000000,5.000000,2.000000,4.000000,7.000000,4.000000,4.000000,6.000000,3.000000,9.000000,5.000000,12.000000,12.000000,14.000000,21.000000,16.000000,2.000000,15.000000,12.000000,17.000000,3.000000,11.000000,3.000000,2.000000,10.000000,5.000000,3.000000,3.000000,2.000000,3.000000,1.000000,1.000000,1.000000,1.000000,1.000000
25%,4551.500000,200315.500000,21.000000,62.000000,67.000000,1457.000000,1.000000,3.000000,2.000000,8.000000,38.000000,30.000000,44.000000,54.000000,30.000000,49.000000,34.000000,31.000000,43.000000,54.000000,57.000000,57.000000,55.000000,56.000000,56.000000,45.000000,58.000000,56.000000,58.000000,33.000000,44.000000,26.000000,38.000000,44.000000,39.000000,51.000000,30.000000,27.000000,24.000000,8.000000,8.000000,8.000000,8.000000,8.000000
50%,9103.000000,221759.000000,25.000000,66.000000,71.000000,1635.000000,1.000000,3.000000,2.000000,17.000000,54.000000,49.000000,56.000000,62.000000,44.000000,61.000000,48.000000,41.000000,56.000000,63.000000,67.000000,67.000000,66.000000,62.000000,66.000000,59.000000,66.000000,66.000000,67.000000,51.000000,59.000000,52.000000,55.000000,55.000000,49.000000,60.000000,53.000000,55.000000,52.000000,11.000000,11.000000,11.000000,11.000000,11.000000
75%,13654.500000,236529.500000,28.000000,71.000000,75.000000,1787.000000,1.000000,3.000000,3.000000,26.000000,64.000000,62.000000,64.000000,68.000000,57.000000,68.000000,62.000000,57.000000,64.000000,69.000000,75.000000,75.000000,74.000000,68.000000,74.000000,68.000000,73.000000,74.000000,74.000000,62.000000,69.000000,64.000000,64.000000,64.000000,60.000000,67.000000,64.000000,66.000000,64.000000,14.000000,14.000000,14.000000,14.000000,14.000000
max,18206.000000,246620.000000,45.000000,94.000000,95.000000,2346.000000,5.000000,5.000000,5.000000,99.000000,93.000000,95.000000,94.000000,93.000000,90.000000,97.000000,94.000000,94.000000,93.000000,96.000000,97.000000,96.000000,96.000000,96.000000,96.000000,95.000000,95.000000,96.000000,97.000000,94.000



Amb la descripció feta dels atributs, podem eliminar directament les columnes que no aporten res a la predicció, aquests són:

``Unnamed: 0:`` El nombre d'identificació en el dataset no aporta informació per la predicció.<br />
``ID:`` El nombre d'identificació en el FIFA no aporta informació en la predicció.<br />
``Name:`` Ja que el nom del jugador com a tal no afecta al preu d'ell en el mercat.<br />
``Photo:`` URL d'una imatge.<br />
``Flag:`` URL d'una imatge.<br />
``Special:`` Si té o no cartes especials no aporta informació en la predicció.<br />
``Club Logo:`` URL d'una imatge.<br />
``Body Type:`` El model de cos 3D en el joc no aporta res a la predicció.<br />
``Real Face:`` Si té o no cara en el FIFA no aporta res a la predicció.<br />
``Jersey Number:`` El nombre de dorsal no aporta informació a la predicció.<br />
``Joined:`` Baix el meu punt de vista no afecta al preu del jugador la quantitat de temps que duu jugant a un mateix club, ja que el que importa són els mèrtits personals no la durabilitat en el club. Pot pertanyer a un club durant molt de temps però no aportar res a aquest.<br />
``Loaned From:`` Baix el meu punt de vista no afecta al preu del jugador per quin equip està cedit, ja que tendrà més importància saber el club al que pertany pel prestigui que pugui obtenir d'ell.<br />


In [5]:
df = df.drop(['Unnamed: 0', 'ID', 'Special', 'Jersey Number', 'Name', 'Photo', 'Flag', 'Club Logo', 'Body Type', 'Real Face', 'Joined', 'Loaned From'], axis=1)
df.head()

,Age,Nationality,Overall,Potential,Club,Value,Wage,Preferred Foot,International Reputation,Weak Foot,Skill Moves,Work Rate,Position,Contract Valid Until,Height,Weight,LS,ST,RS,LW,LF,CF,RF,RW,LAM,CAM,RAM,LM,LCM,CM,RCM,RM,LWB,LDM,CDM,RDM,RWB,LB,LCB,CB,RCB,RB,Crossing,Finishing,HeadingAccuracy,ShortPassing,Volleys,Dribbling,Curve,FKAccuracy,LongPassing,BallControl,Acceleration,SprintSpeed,Agility,Reactions,Balance,ShotPower,Jumping,Stamina,Strength,LongShots,Aggression,Interceptions,Positioning,Vision,Penalties,Composure,Marking,StandingTackle,SlidingTackle,GKDiving,GKHandling,GKKicking,GKPositioning,GKReflexes,Release Clause
0,31,Argentina,94,94,FC Barcelona,€110.5M,€565K,Left,5.0,4.0,4.0,Medium/ Medium,RF,2021,5'7,159lbs,88+2,88+2,88+2,92+2,93+2,93+2,93+2,92+2,93+2,93+2,93+2,91+2,84+2,84+2,84+2,91+2,64+2,61+2,61+2,61+2,64+2,59+2,47+2,47+2,47+2,59+2,84.0,95.0,70.0,90.0,86.0,97.0,93.0,94.0,87.0,96.0,91.0,86.0,91.0,95.0,95.0,85.0,68.0,72.0,59.0,94.0,48.0,22.0,94.0,94.0,75.0,96.0,33.0,28.0,26.0,6.0,11.0,15.0,14.0,8.0,€226.5M
1,33,Portugal,94,94,Juventus,€77M,€405K,Right,5.0,4.0,5.0,High/ Low,ST,2022,6'2,183lbs,91+3,91+3,91+3,89+3,90+3,90+3,90+3,89+3,88+3,88+3,88+3,88+3,81+3,81+3,81+3,88+3,65+3,61+3,61+3,61+3,65+3,61+3,53+3,53+3,53+3,61+3,84.0,94.0,89.0,81.0,87.0,88.0,81.0,76.0,77.0,94.0,89.0,91.0,87.0,96.0,70.0,95.0,95.0,88.0,79.0,93.0,63.0,29.0,95.0,82.0,85.0,95.0,28.0,31.0,23.0,7.0,11.0,15.0,14.0,11.0,€127.1M
2,26,Brazil,92,93,Paris Saint-Germain,€118.5M,€290K,Right,5.0,5.0,5.0,High/ Medium,LW,2022,5'9,150lbs,84+3,84+3,84+3,89+3,89+3,89+3,89+3,89+3,89+3,89+3,89+3,88+3,81+3,81+3,81+3,88+3,65+3,60+3,60+3,60+3,65+3,60+3,47+3,47+3,47+3,60+3,79.0,87.0,62.0,84.0,84.0,96.0,88.0,87.0,78.0,95.0,94.0,90.0,96.0,94.0,84.0,80.0,61.0,81.0,49.0,82.0,56.0,36.0,89.0,87.0,81.0,94.0,27.0,24.0,33.0,9.0,9.0,15.0,15.0,11.0,€228.1M
3,27,Spain,91,93,Manchester United,€72M,€260K,Right,4.0,3.0,1.0,Medium/ Medium,GK,2020,6'4,168lbs,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,17.0,13.0,21.0,50.0,13.0,18.0,21.0,19.0,51.0,42.0,57.0,58.0,60.0,90.0,43.0,31.0,67.0,43.0,64.0,12.0,38.0,30.0,12.0,68.0,40.0,68.0,15.0,21.0,13.0,90.0,85.0,87.0,88.0,94.0,€138.6M
4,27,Belgium,91,92,Manchester City,€102M,€355K,Right,4.0,5.0,4.0,High/ High,RCM,2023,5'11,154lbs,82+3,82+3,82+3,87+3,87+3,87+3,87+3,87+3,88+3,88+3,88+3,88+3,87+3,87+3,87+3,88+3,77+3,77+3,77+3,77+3,77+3,73+3,66+3,66+3,66+3,73+3,93.0,82.0,55.0,92.0,82.0,86.0,85.0,83.0,91.0,91.0,78.0,76.0,79.0,91.0,77.0,91.0,63.0,90.0,75.0,91.0,76.0,61.0,87.0,94.0,79.0,88.0,68.0,58.0,51.0,15.0,13.0,5.0,10.0,13.0,€196.4M


D'altre banda la resta de columnes es conserveran ja que afecten al valor del jugador per:

``Age:`` Si un jugador té una certa edat, normalment no tendrà el mateix rendiment que un jove, per tant convé mantenir aquest valor.<br />
``Nationality:`` Pot afectar ja que si ve d'un mercat amb tradició futbolística sol ser més valorat que un provinent d'un país poc interessat per aquest esport.<br />
``Overall`` La mitja dels seus atributs clarament s'ha de mantenir, ja que indica globalment quant de bo és un jugador.<br />
``Potential:`` La prospecció de futur d'un jugador interesserà als clubs per rentabilitzar l'inversió, per tant pagaran més per un jugador que té un potencial elevat.<br />
``Club:`` L'equip al que pertany un jugador li aporta prestigui.<br />
``Value:`` Preu de mercat, atribut a fer la predicció.<br />
``Wage:`` Si un jugador cobra molt, es suposa que és perquè és bo o perquè el club no vol que es vagi del club, ambdós casos provocaran un augment del seu preu.bueno<br />
``Preferred Foot:`` En general hi ha menys jugadors esquerrans, els quals son necessaris en les posicions situades en el costat esquerra del camp, aquest menor nombre d'esquerrans farà que aquests tinguin un preu superior respecte a un dretà per ocupar la mateixa posició.<br />
``International Reputation:`` Si té una bona reputació internacional serà més desitjat i a causa d'això més valorat.<br />
``Weak Foot:`` Es valora l'habilitat del jugador amb el peu dolent per tant s'ha de mantenir.<br />
``Skill Moves:`` Es valora l'habilitat del jugador en fer filigranes per tant s'ha de mantenir.<br />
``Work Rate:`` Es valora el ritme de treball, com més elevat millor rendiment i per conseqüència major preu.<br />
``Position:`` Hi ha posicions més valorades dins el camp per l'impacte que tenen en el partit, per tant els jugadors que ocupen aquestes són més valorats.<br />
``Contract Valid Until:``Si un jugador li queden molts anys de contracte, un segon equip que el vulgui comprar, haurà de pagar un preu major per tenir-ho.<br />
``Height:`` Un jugador alt pot ser millor per fer contracaps o com a porter, per tant afecterà al preu.<br />
``Weight:`` Un jugador en mal estat físic tendrà un preu inferior.<br />
``LS:`` Indica l'habilitat del jugador en la posició de "Left striker", afectarà al preu.<br />
``ST:`` Indica l'habilitat del jugador en la posició de "Striker", afectarà al preu.<br />
``RS:`` Indica l'habilitat del jugador en la posició de "Right striker", afectarà al preu.<br />
``LW:`` Indica l'habilitat del jugador en la posició de "Left winger", afectarà al preu.<br />
``LF:`` Indica l'habilitat del jugador en la posició de "Left forward", afectarà al preu.<br />
``CF:`` Indica l'habilitat del jugador en la posició de "Central forward", afectarà al preu.<br />
``RF:`` Indica l'habilitat del jugador en la posició de "Right forward", afectarà al preu.<br />
``RW:`` Indica l'habilitat del jugador en la posició de "Right winger", afectarà al preu.<br />
``LAM:`` Indica l'habilitat del jugador en la posició de "Left attacking midfielder", afectarà al preu.<br />
``CAM:`` Indica l'habilitat del jugador en la posició de "Central attacking midfielder", afectarà al preu.<br />
``RAM:`` Indica l'habilitat del jugador en la posició de "Right attacking midfielder", afectarà al preu.<br />
``LM:`` Indica l'habilitat del jugador en la posició de "Left midfielder", afectarà al preu.<br />
``LCM:`` Indica l'habilitat del jugador en la posició de "Left central midfielder", afectarà al preu.<br />
``CM:`` Indica l'habilitat del jugador en la posició de "Central midfielder", afectarà al preu.<br />
``RCM:`` Indica l'habilitat del jugador en la posició de "Right central midfielder", afectarà al preu.<br />
``RM:`` Indica l'habilitat del jugador en la posició de "Right midfielder", afectarà al preu.<br />
``LWB:`` Indica l'habilitat del jugador en la posició de "Left winger back".<br />
``LDM:`` Indica l'habilitat del jugador en la posició de "Left defense midfielder", afectarà al preu.<br />
``CDM:`` Indica l'habilitat del jugador en la posició de "Central defense midfielder", afectarà al preu.<br />
``RDM:`` Indica l'habilitat del jugador en la posició de "Right defense midfielder", afectarà al preu.<br />
``RWB:`` Indica l'habilitat del jugador en la posició de "Right winger back", afectarà al preu.<br />
``LB:`` Indica l'habilitat del jugador en la posició de "Left back", afectarà al preu.<br />
``LCB:`` Indica l'habilitat del jugador en la posició de "Left central back", afectarà al preu.<br />
``CB:`` Indica l'habilitat del jugador en la posició de "Central back", afectarà al preu.<br />
``RCB:`` Indica l'habilitat del jugador en la posició de "Right central back", afectarà al preu.<br />
``RB:`` Indica l'habilitat del jugador en la posició de "Right back", afectarà al preu.<br />
``Crossing:`` (\*).<br />
``Finishing:`` (\*).<br />
``HeadingAccuracy:`` (\*).<br />
``ShortPassing:`` (\*).<br />
``Volleys Dribbling:`` (\*).<br />
``Curve:`` (\*).<br />
``FKAccuracy:`` (\*).<br />
``LongPassing:`` (\*).<br />
``BallControl:`` (\*).<br />
``Acceleration:`` (\*).<br />
``SprintSpeed:`` (\*).<br />
``Agility Reactions:`` (\*).<br />
``Balance:`` (\*).<br />
``ShotPower:`` (\*).<br />
``Jumping Stamina Strength:`` (\*).<br />
``Long Shots:`` (\*).<br />
``Aggression:`` (\*).<br />
``Interceptions:`` (\*).<br />
``Positioning:`` (\*).<br />
``Vision:`` (\*).<br />
``Penalties:`` (\*).<br />
``Composure:`` (\*).<br />
``Marking Standing Tackle:`` (\*).<br />
``SlidingTackle:`` (\*).<br />
``GKDiving:`` (\*).<br />
``GKHandling:`` (\*).<br />
``GKKicking:`` (\*).<br />
``GKPositioning:`` (\*).<br />
``GKReflexes:`` (\*).<br />
``Release Clause:``El preu que ha de pagar un club extern per cancel·lar el contracte del jugador amb el club actual afectarà al preu directament.<br />

(\*): Valoració numèrica de les habilitats futbolístiques del jugador, afecten al preu ja que defineix la qualitat d'aquest.

### Eliminació de NaNS

Els NaNs ens impedeixen realitzar estimacións correctes, per tant es important no tenir-ne a l'hora de realitzar les prediccions. Per saber quines columnes tenen NaNs realitzarem la seguent instrucció:

In [6]:
df.columns[df.isna().any()].tolist()

['Club',
 'Preferred Foot',
 'International Reputation',
 'Weak Foot',
 'Skill Moves',
 'Work Rate',
 'Position',
 'Contract Valid Until',
 'Height',
 'Weight',
 'LS',
 'ST',
 'RS',
 'LW',
 'LF',
 'CF',
 'RF',
 'RW',
 'LAM',
 'CAM',
 'RAM',
 'LM',
 'LCM',
 'CM',
 'RCM',
 'RM',
 'LWB',
 'LDM',
 'CDM',
 'RDM',
 'RWB',
 'LB',
 'LCB',
 'CB',
 'RCB',
 'RB',
 'Crossing',
 'Finishing',
 'HeadingAccuracy',
 'ShortPassing',
 'Volleys',
 'Dribbling',
 'Curve',
 'FKAccuracy',
 'LongPassing',
 'BallControl',
 'Acceleration',
 'SprintSpeed',
 'Agility',
 'Reactions',
 'Balance',
 'ShotPower',
 'Jumping',
 'Stamina',
 'Strength',
 'LongShots',
 'Aggression',
 'Interceptions',
 'Positioning',
 'Vision',
 'Penalties',
 'Composure',
 'Marking',
 'StandingTackle',
 'SlidingTackle',
 'GKDiving',
 'GKHandling',
 'GKKicking',
 'GKPositioning',
 'GKReflexes',
 'Release Clause']

In [7]:
df[df.isna().any(axis=1)]


,Age,Nationality,Overall,Potential,Club,Value,Wage,Preferred Foot,International Reputation,Weak Foot,Skill Moves,Work Rate,Position,Contract Valid Until,Height,Weight,LS,ST,RS,LW,LF,CF,RF,RW,LAM,CAM,RAM,LM,LCM,CM,RCM,RM,LWB,LDM,CDM,RDM,RWB,LB,LCB,CB,RCB,RB,Crossing,Finishing,HeadingAccuracy,ShortPassing,Volleys,Dribbling,Curve,FKAccuracy,LongPassing,BallControl,Acceleration,SprintSpeed,Agility,Reactions,Balance,ShotPower,Jumping,Stamina,Strength,LongShots,Aggression,Interceptions,Positioning,Vision,Penalties,Composure,Marking,StandingTackle,SlidingTackle,GKDiving,GKHandling,GKKicking,GKPositioning,GKReflexes,Release Clause
3,27,Spain,91,93,Manchester United,€72M,€260K,Right,4.0,3.0,1.0,Medium/ Medium,GK,2020,6'4,168lbs,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,17.0,13.0,21.0,50.0,13.0,18.0,21.0,19.0,51.0,42.0,57.0,58.0,60.0,90.0,43.0,31.0,67.0,43.0,64.0,12.0,38.0,30.0,12.0,68.0,40.0,68.0,15.0,21.0,13.0,90.0,85.0,87.0,88.0,94.0,€138.6M
9,25,Slovenia,90,93,Atlético Madrid,€68M,€94K,Right,3.0,3.0,1.0,Medium/ Medium,GK,2021,6'2,192lbs,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,13.0,11.0,15.0,29.0,13.0,12.0,13.0,14.0,26.0,16.0,43.0,60.0,67.0,86.0,49.0,22.0,76.0,41.0,78.0,12.0,34.0,19.0,11.0,70.0,11.0,70.0,27.0,12.0,18.0,86.0,92.0,78.0,88.0,89.0,€144.5M
18,26,Germany,89,92,FC Barcelona,€58M,€240K,Right,3.0,4.0,1.0,Medium/ Medium,GK,2022,6'2,187lbs,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,15.0,14.0,11.0,36.0,14.0,17.0,18.0,12.0,42.0,18.0,38.0,50.0,37.0,85.0,43.0,22.0,79.0,35.0,79.0,10.0,43.0,22.0,11.0,69.0,25.0,69.0,25.0,13.0,10.0,87.0,85.0,88.0,85.0,90.0,€123.3M
19,26,Belgium,89,90,Real Madrid,€53.5M,€240K,Left,4.0,2.0,1.0,Medium/ Medium,GK,2024,6'6,212lbs,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,14.0,14.0,13.0,33.0,12.0,13.0,19.0,20.0,35.0,23.0,46.0,52.0,61.0,84.0,45.0,36.0,68.0,38.0,70.0,17.0,23.0,15.0,13.0,44.0,27.0,66.0,20.0,18.0,16.0,85.0,91.0,72.0,86.0,88.0,€113.7M
22,32,Germany,89,89,FC Bayern München,€38M,€130K,Right,5.0,4.0,1.0,Medium/ Medium,GK,2021,6'4,203lbs,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,15.0,13.0,25.0,55.0,11.0,30.0,14.0,11.0,59.0,48.0,54.0,60.0,51.0,84.0,35.0,25.0,77.0,43.0,80.0,16.0,29.0,30.0,12.0,70.0,47.0,70.0,17.0,10.0,11.0,90.0,86.0,91.0,87.0,87.0,€62.7M
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18178,18,Sweden,48,65,Dalkurd FF,€50K,€1K,Right,1.0,2.0,1.0,Medium/ Medium,GK,2018,6'0,176lbs,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10.0,7.0,11.0,22.0,6.0,8.0,10.0,8.0,20.0,12.0,24.0,28.0,33.0,40.0,29.0,15.0,48.0,20.0,38.0,6.0,20.0,10.0,7.0,18.0,12.0,28.0,16.0,11.0,10.0,47.0,46.0,50.0,45.0,51.0,€94K
18180,22,Scotland,48,58,St. Johnstone FC,€40K,€1K,Right,1.0,2.0,1.0,Medium/ Medium,GK,2019,6'1,172lbs,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,19.0,15.0,15.0,26.0,14.0,13.0,12.0,19.0,19.0,25.0,40.0,42.0,45.0,47.0,52.0,25.0,55.0,33.0,58.0,20.0,20.0,29.0,13.0,20.0,29.0,28.0,12.0,15.0,16.0,45.0,49.0,50.0,50.0,45.0,€78K
18183,44,England,48,48,Cambridge United,€0,€1K,Right,1.0,2.0,1.0,Medium/ Medium,GK,2023,6'2,190lbs,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,11.0,12.0,12.0,18.0,11.0,11.0,12.0,13.0,13.0,16.0,15.0,17.0,31.0,48.0,31.0,23.0,45.0,22.0,44.0,12.0,24.0,15.0,10.0,11.0,22.0,56.0,15.0,15.0,13.0,45.0,48.0,44.0,49.0,46.0,NaN
18194,18,Italy,47,65,Lecce,€50K,€1

El club al que pertanyen pot causar una gran diferencia al valor d'un jugador per causes com pugui ser el prestigui que aporta la pertenyença, per tant s'eliminaran els jugadors que no disposen d'aquest valor:

In [8]:
df = df.dropna(subset = ['Club'])
df.head()

,Age,Nationality,Overall,Potential,Club,Value,Wage,Preferred Foot,International Reputation,Weak Foot,Skill Moves,Work Rate,Position,Contract Valid Until,Height,Weight,LS,ST,RS,LW,LF,CF,RF,RW,LAM,CAM,RAM,LM,LCM,CM,RCM,RM,LWB,LDM,CDM,RDM,RWB,LB,LCB,CB,RCB,RB,Crossing,Finishing,HeadingAccuracy,ShortPassing,Volleys,Dribbling,Curve,FKAccuracy,LongPassing,BallControl,Acceleration,SprintSpeed,Agility,Reactions,Balance,ShotPower,Jumping,Stamina,Strength,LongShots,Aggression,Interceptions,Positioning,Vision,Penalties,Composure,Marking,StandingTackle,SlidingTackle,GKDiving,GKHandling,GKKicking,GKPositioning,GKReflexes,Release Clause
0,31,Argentina,94,94,FC Barcelona,€110.5M,€565K,Left,5.0,4.0,4.0,Medium/ Medium,RF,2021,5'7,159lbs,88+2,88+2,88+2,92+2,93+2,93+2,93+2,92+2,93+2,93+2,93+2,91+2,84+2,84+2,84+2,91+2,64+2,61+2,61+2,61+2,64+2,59+2,47+2,47+2,47+2,59+2,84.0,95.0,70.0,90.0,86.0,97.0,93.0,94.0,87.0,96.0,91.0,86.0,91.0,95.0,95.0,85.0,68.0,72.0,59.0,94.0,48.0,22.0,94.0,94.0,75.0,96.0,33.0,28.0,26.0,6.0,11.0,15.0,14.0,8.0,€226.5M
1,33,Portugal,94,94,Juventus,€77M,€405K,Right,5.0,4.0,5.0,High/ Low,ST,2022,6'2,183lbs,91+3,91+3,91+3,89+3,90+3,90+3,90+3,89+3,88+3,88+3,88+3,88+3,81+3,81+3,81+3,88+3,65+3,61+3,61+3,61+3,65+3,61+3,53+3,53+3,53+3,61+3,84.0,94.0,89.0,81.0,87.0,88.0,81.0,76.0,77.0,94.0,89.0,91.0,87.0,96.0,70.0,95.0,95.0,88.0,79.0,93.0,63.0,29.0,95.0,82.0,85.0,95.0,28.0,31.0,23.0,7.0,11.0,15.0,14.0,11.0,€127.1M
2,26,Brazil,92,93,Paris Saint-Germain,€118.5M,€290K,Right,5.0,5.0,5.0,High/ Medium,LW,2022,5'9,150lbs,84+3,84+3,84+3,89+3,89+3,89+3,89+3,89+3,89+3,89+3,89+3,88+3,81+3,81+3,81+3,88+3,65+3,60+3,60+3,60+3,65+3,60+3,47+3,47+3,47+3,60+3,79.0,87.0,62.0,84.0,84.0,96.0,88.0,87.0,78.0,95.0,94.0,90.0,96.0,94.0,84.0,80.0,61.0,81.0,49.0,82.0,56.0,36.0,89.0,87.0,81.0,94.0,27.0,24.0,33.0,9.0,9.0,15.0,15.0,11.0,€228.1M
3,27,Spain,91,93,Manchester United,€72M,€260K,Right,4.0,3.0,1.0,Medium/ Medium,GK,2020,6'4,168lbs,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,17.0,13.0,21.0,50.0,13.0,18.0,21.0,19.0,51.0,42.0,57.0,58.0,60.0,90.0,43.0,31.0,67.0,43.0,64.0,12.0,38.0,30.0,12.0,68.0,40.0,68.0,15.0,21.0,13.0,90.0,85.0,87.0,88.0,94.0,€138.6M
4,27,Belgium,91,92,Manchester City,€102M,€355K,Right,4.0,5.0,4.0,High/ High,RCM,2023,5'11,154lbs,82+3,82+3,82+3,87+3,87+3,87+3,87+3,87+3,88+3,88+3,88+3,88+3,87+3,87+3,87+3,88+3,77+3,77+3,77+3,77+3,77+3,73+3,66+3,66+3,66+3,73+3,93.0,82.0,55.0,92.0,82.0,86.0,85.0,83.0,91.0,91.0,78.0,76.0,79.0,91.0,77.0,91.0,63.0,90.0,75.0,91.0,76.0,61.0,87.0,94.0,79.0,88.0,68.0,58.0,51.0,15.0,13.0,5.0,10.0,13.0,€196.4M


Es tornen a mirar quins jugadors segueixen tenint valors Nan:

In [9]:
df.columns[df.isna().any()].tolist()

['Preferred Foot',
 'International Reputation',
 'Weak Foot',
 'Skill Moves',
 'Work Rate',
 'Position',
 'Contract Valid Until',
 'Height',
 'Weight',
 'LS',
 'ST',
 'RS',
 'LW',
 'LF',
 'CF',
 'RF',
 'RW',
 'LAM',
 'CAM',
 'RAM',
 'LM',
 'LCM',
 'CM',
 'RCM',
 'RM',
 'LWB',
 'LDM',
 'CDM',
 'RDM',
 'RWB',
 'LB',
 'LCB',
 'CB',
 'RCB',
 'RB',
 'Crossing',
 'Finishing',
 'HeadingAccuracy',
 'ShortPassing',
 'Volleys',
 'Dribbling',
 'Curve',
 'FKAccuracy',
 'LongPassing',
 'BallControl',
 'Acceleration',
 'SprintSpeed',
 'Agility',
 'Reactions',
 'Balance',
 'ShotPower',
 'Jumping',
 'Stamina',
 'Strength',
 'LongShots',
 'Aggression',
 'Interceptions',
 'Positioning',
 'Vision',
 'Penalties',
 'Composure',
 'Marking',
 'StandingTackle',
 'SlidingTackle',
 'GKDiving',
 'GKHandling',
 'GKKicking',
 'GKPositioning',
 'GKReflexes',
 'Release Clause']

Veim que l'atribut "Release Clause" té valors NaNs, es suposarà que si el jugador no té cap clausula de recesió aquesta serà 0€ per tant:

In [10]:
df["Release Clause"] = df["Release Clause"].fillna("€0")

Es miren de nou els valors NaN:

In [11]:
df.columns[df.isna().any()].tolist()

['Preferred Foot',
 'International Reputation',
 'Weak Foot',
 'Skill Moves',
 'Work Rate',
 'Position',
 'Contract Valid Until',
 'Height',
 'Weight',
 'LS',
 'ST',
 'RS',
 'LW',
 'LF',
 'CF',
 'RF',
 'RW',
 'LAM',
 'CAM',
 'RAM',
 'LM',
 'LCM',
 'CM',
 'RCM',
 'RM',
 'LWB',
 'LDM',
 'CDM',
 'RDM',
 'RWB',
 'LB',
 'LCB',
 'CB',
 'RCB',
 'RB',
 'Crossing',
 'Finishing',
 'HeadingAccuracy',
 'ShortPassing',
 'Volleys',
 'Dribbling',
 'Curve',
 'FKAccuracy',
 'LongPassing',
 'BallControl',
 'Acceleration',
 'SprintSpeed',
 'Agility',
 'Reactions',
 'Balance',
 'ShotPower',
 'Jumping',
 'Stamina',
 'Strength',
 'LongShots',
 'Aggression',
 'Interceptions',
 'Positioning',
 'Vision',
 'Penalties',
 'Composure',
 'Marking',
 'StandingTackle',
 'SlidingTackle',
 'GKDiving',
 'GKHandling',
 'GKKicking',
 'GKPositioning',
 'GKReflexes']

Si ens fixam, la majoria de jugadors que tenen els atributs: *LS, ST, RS, LW, LF, CF, RF, RW, LAM, CAM, RAM, LM, LCM, CM, RCM, RM, LWB, LDM, CDM, RDM, RWB, LB, LCB, CB, RCB i RB*. Són porters.

Per tal de resoldre-ho es pot fer de dues maneres: eliminar aquests jugadors o donar un valor a aquestes variables. Com també interessa poder realitzar una predicció sobre els preus dels porters, es procedirà amb la segona opció. Ara ens queda determinar quin valor es reemplaçarà pels NaNs. Si es posés a 0, podrien sortir prediccions irregulars, com en el cas de dos jugadors amb el mateix preu, un porter i un davanter, el primer amb tot 0's als valors de posició i el segon amb nombres alts. Per tant el més simple per assignar un valor és amb la mitja feta amb els jugadors no porters.

Si ens fixam, aquestes columnes tenen els valors en format String ("X+Y") que es passaran a valor numèric més endavant, per tant separarem els porters de la resta de jugadors per resoldre aquests NaNs una vegada tinguin valors numèrics aquests atributs.

In [12]:
gk = df[df['Position']  == "GK"]
indexGK = df[df['Position']  == "GK"].index
df.drop(indexGK , inplace=True)
gk.head()

,Age,Nationality,Overall,Potential,Club,Value,Wage,Preferred Foot,International Reputation,Weak Foot,Skill Moves,Work Rate,Position,Contract Valid Until,Height,Weight,LS,ST,RS,LW,LF,CF,RF,RW,LAM,CAM,RAM,LM,LCM,CM,RCM,RM,LWB,LDM,CDM,RDM,RWB,LB,LCB,CB,RCB,RB,Crossing,Finishing,HeadingAccuracy,ShortPassing,Volleys,Dribbling,Curve,FKAccuracy,LongPassing,BallControl,Acceleration,SprintSpeed,Agility,Reactions,Balance,ShotPower,Jumping,Stamina,Strength,LongShots,Aggression,Interceptions,Positioning,Vision,Penalties,Composure,Marking,StandingTackle,SlidingTackle,GKDiving,GKHandling,GKKicking,GKPositioning,GKReflexes,Release Clause
3,27,Spain,91,93,Manchester United,€72M,€260K,Right,4.0,3.0,1.0,Medium/ Medium,GK,2020,6'4,168lbs,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,17.0,13.0,21.0,50.0,13.0,18.0,21.0,19.0,51.0,42.0,57.0,58.0,60.0,90.0,43.0,31.0,67.0,43.0,64.0,12.0,38.0,30.0,12.0,68.0,40.0,68.0,15.0,21.0,13.0,90.0,85.0,87.0,88.0,94.0,€138.6M
9,25,Slovenia,90,93,Atlético Madrid,€68M,€94K,Right,3.0,3.0,1.0,Medium/ Medium,GK,2021,6'2,192lbs,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,13.0,11.0,15.0,29.0,13.0,12.0,13.0,14.0,26.0,16.0,43.0,60.0,67.0,86.0,49.0,22.0,76.0,41.0,78.0,12.0,34.0,19.0,11.0,70.0,11.0,70.0,27.0,12.0,18.0,86.0,92.0,78.0,88.0,89.0,€144.5M
18,26,Germany,89,92,FC Barcelona,€58M,€240K,Right,3.0,4.0,1.0,Medium/ Medium,GK,2022,6'2,187lbs,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,15.0,14.0,11.0,36.0,14.0,17.0,18.0,12.0,42.0,18.0,38.0,50.0,37.0,85.0,43.0,22.0,79.0,35.0,79.0,10.0,43.0,22.0,11.0,69.0,25.0,69.0,25.0,13.0,10.0,87.0,85.0,88.0,85.0,90.0,€123.3M
19,26,Belgium,89,90,Real Madrid,€53.5M,€240K,Left,4.0,2.0,1.0,Medium/ Medium,GK,2024,6'6,212lbs,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,14.0,14.0,13.0,33.0,12.0,13.0,19.0,20.0,35.0,23.0,46.0,52.0,61.0,84.0,45.0,36.0,68.0,38.0,70.0,17.0,23.0,15.0,13.0,44.0,27.0,66.0,20.0,18.0,16.0,85.0,91.0,72.0,86.0,88.0,€113.7M
22,32,Germany,89,89,FC Bayern München,€38M,€130K,Right,5.0,4.0,1.0,Medium/ Medium,GK,2021,6'4,203lbs,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,15.0,13.0,25.0,55.0,11.0,30.0,14.0,11.0,59.0,48.0,54.0,60.0,51.0,84.0,35.0,25.0,77.0,43.0,80.0,16.0,29.0,30.0,12.0,70.0,47.0,70.0,17.0,10.0,11.0,90.0,86.0,91.0,87.0,87.0,€62.7M


In [13]:
df.head()

,Age,Nationality,Overall,Potential,Club,Value,Wage,Preferred Foot,International Reputation,Weak Foot,Skill Moves,Work Rate,Position,Contract Valid Until,Height,Weight,LS,ST,RS,LW,LF,CF,RF,RW,LAM,CAM,RAM,LM,LCM,CM,RCM,RM,LWB,LDM,CDM,RDM,RWB,LB,LCB,CB,RCB,RB,Crossing,Finishing,HeadingAccuracy,ShortPassing,Volleys,Dribbling,Curve,FKAccuracy,LongPassing,BallControl,Acceleration,SprintSpeed,Agility,Reactions,Balance,ShotPower,Jumping,Stamina,Strength,LongShots,Aggression,Interceptions,Positioning,Vision,Penalties,Composure,Marking,StandingTackle,SlidingTackle,GKDiving,GKHandling,GKKicking,GKPositioning,GKReflexes,Release Clause
0,31,Argentina,94,94,FC Barcelona,€110.5M,€565K,Left,5.0,4.0,4.0,Medium/ Medium,RF,2021,5'7,159lbs,88+2,88+2,88+2,92+2,93+2,93+2,93+2,92+2,93+2,93+2,93+2,91+2,84+2,84+2,84+2,91+2,64+2,61+2,61+2,61+2,64+2,59+2,47+2,47+2,47+2,59+2,84.0,95.0,70.0,90.0,86.0,97.0,93.0,94.0,87.0,96.0,91.0,86.0,91.0,95.0,95.0,85.0,68.0,72.0,59.0,94.0,48.0,22.0,94.0,94.0,75.0,96.0,33.0,28.0,26.0,6.0,11.0,15.0,14.0,8.0,€226.5M
1,33,Portugal,94,94,Juventus,€77M,€405K,Right,5.0,4.0,5.0,High/ Low,ST,2022,6'2,183lbs,91+3,91+3,91+3,89+3,90+3,90+3,90+3,89+3,88+3,88+3,88+3,88+3,81+3,81+3,81+3,88+3,65+3,61+3,61+3,61+3,65+3,61+3,53+3,53+3,53+3,61+3,84.0,94.0,89.0,81.0,87.0,88.0,81.0,76.0,77.0,94.0,89.0,91.0,87.0,96.0,70.0,95.0,95.0,88.0,79.0,93.0,63.0,29.0,95.0,82.0,85.0,95.0,28.0,31.0,23.0,7.0,11.0,15.0,14.0,11.0,€127.1M
2,26,Brazil,92,93,Paris Saint-Germain,€118.5M,€290K,Right,5.0,5.0,5.0,High/ Medium,LW,2022,5'9,150lbs,84+3,84+3,84+3,89+3,89+3,89+3,89+3,89+3,89+3,89+3,89+3,88+3,81+3,81+3,81+3,88+3,65+3,60+3,60+3,60+3,65+3,60+3,47+3,47+3,47+3,60+3,79.0,87.0,62.0,84.0,84.0,96.0,88.0,87.0,78.0,95.0,94.0,90.0,96.0,94.0,84.0,80.0,61.0,81.0,49.0,82.0,56.0,36.0,89.0,87.0,81.0,94.0,27.0,24.0,33.0,9.0,9.0,15.0,15.0,11.0,€228.1M
4,27,Belgium,91,92,Manchester City,€102M,€355K,Right,4.0,5.0,4.0,High/ High,RCM,2023,5'11,154lbs,82+3,82+3,82+3,87+3,87+3,87+3,87+3,87+3,88+3,88+3,88+3,88+3,87+3,87+3,87+3,88+3,77+3,77+3,77+3,77+3,77+3,73+3,66+3,66+3,66+3,73+3,93.0,82.0,55.0,92.0,82.0,86.0,85.0,83.0,91.0,91.0,78.0,76.0,79.0,91.0,77.0,91.0,63.0,90.0,75.0,91.0,76.0,61.0,87.0,94.0,79.0,88.0,68.0,58.0,51.0,15.0,13.0,5.0,10.0,13.0,€196.4M
5,27,Belgium,91,91,Chelsea,€93M,€340K,Right,4.0,4.0,4.0,High/ Medium,LF,2020,5'8,163lbs,83+3,83+3,83+3,89+3,88+3,88+3,88+3,89+3,89+3,89+3,89+3,89+3,82+3,82+3,82+3,89+3,66+3,63+3,63+3,63+3,66+3,60+3,49+3,49+3,49+3,60+3,81.0,84.0,61.0,89.0,80.0,95.0,83.0,79.0,83.0,94.0,94.0,88.0,95.0,90.0,94.0,82.0,56.0,83.0,66.0,80.0,54.0,41.0,87.0,89.0,86.0,91.0,34.0,27.0,22.0,11.0,12.0,6.0,8.0,8.0,€172.1M


Es miren de nous els valors NaN

In [14]:
df.columns[df.isna().any()].tolist()

['Preferred Foot',
 'International Reputation',
 'Weak Foot',
 'Skill Moves',
 'Work Rate',
 'Position',
 'Contract Valid Until',
 'Height',
 'Weight',
 'LS',
 'ST',
 'RS',
 'LW',
 'LF',
 'CF',
 'RF',
 'RW',
 'LAM',
 'CAM',
 'RAM',
 'LM',
 'LCM',
 'CM',
 'RCM',
 'RM',
 'LWB',
 'LDM',
 'CDM',
 'RDM',
 'RWB',
 'LB',
 'LCB',
 'CB',
 'RCB',
 'RB',
 'Crossing',
 'Finishing',
 'HeadingAccuracy',
 'ShortPassing',
 'Volleys',
 'Dribbling',
 'Curve',
 'FKAccuracy',
 'LongPassing',
 'BallControl',
 'Acceleration',
 'SprintSpeed',
 'Agility',
 'Reactions',
 'Balance',
 'ShotPower',
 'Jumping',
 'Stamina',
 'Strength',
 'LongShots',
 'Aggression',
 'Interceptions',
 'Positioning',
 'Vision',
 'Penalties',
 'Composure',
 'Marking',
 'StandingTackle',
 'SlidingTackle',
 'GKDiving',
 'GKHandling',
 'GKKicking',
 'GKPositioning',
 'GKReflexes']

In [15]:
df[df.isna().any(axis=1)]

,Age,Nationality,Overall,Potential,Club,Value,Wage,Preferred Foot,International Reputation,Weak Foot,Skill Moves,Work Rate,Position,Contract Valid Until,Height,Weight,LS,ST,RS,LW,LF,CF,RF,RW,LAM,CAM,RAM,LM,LCM,CM,RCM,RM,LWB,LDM,CDM,RDM,RWB,LB,LCB,CB,RCB,RB,Crossing,Finishing,HeadingAccuracy,ShortPassing,Volleys,Dribbling,Curve,FKAccuracy,LongPassing,BallControl,Acceleration,SprintSpeed,Agility,Reactions,Balance,ShotPower,Jumping,Stamina,Strength,LongShots,Aggression,Interceptions,Positioning,Vision,Penalties,Composure,Marking,StandingTackle,SlidingTackle,GKDiving,GKHandling,GKKicking,GKPositioning,GKReflexes,Release Clause
13236,33,Scotland,62,62,Rochdale,€120K,€1K,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,€0
13237,29,Nicaragua,62,62,Boyacá Chicó FC,€300K,€1K,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,€0
13238,35,England,62,62,Notts County,€140K,€3K,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,€0
13239,20,Italy,62,72,Brescia,€425K,€1K,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,€0
13240,24,England,62,66,Hamilton Academical FC,€400K,€1K,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,€0
13241,21,Poland,62,72,Śląsk Wrocław,€425K,€1K,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,€0
13242,23,Argentina,62,70,Club Atlético Aldosivi,€450K,€2K,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,€0
13243,19,England,62,78,Everton,€600K,€5K,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,€0
13244,30,Denmark,62,62,Hobro IK,€230K,€2K,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,€0
13245,21,Brazil,62,69,HJK Helsinki,€425K,€1K,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,€0


Es pot veure com hi ha jugadors que tenen una gran quantitat d'atributs fora assignar. Com hi ha massa dades fora valor, s'eliminaran aquests jugadors ja que no aporten res a la predicció per la manca d'informació.

In [16]:
df = df.dropna(subset = ['Position'])
df.columns[df.isna().any()].tolist()

[]

Ja no queden jugadors amb valors NaN.

### Conversió de valors numèrics expresats en Strings a numèrics

En aquest dataset existeixen molts valors numèrics expresats en forma d'String pugui ser per haver afegit alguna mena de prefixe o sufixe en les dades, com volem dades purament numèriques, aquest prefixe o sufixe s'haurà d'eliminar.

Els valors representats en *LS, ST, RS, LW, LF, CF, RF, RW, LAM, CAM, RAM, LM, LCM, CM, RCM, RM, LWB, LDM, CDM, RDM, RWB, LB, LCB, CB, RCB i RB* estan amb el format: "Nombre base + extra", aquest extra determinat per la composició de la plantilla, com ens interesen mirar als jugadors individualment, es deixarà unicament el nombre base. 




In [17]:
def to_base(x):
    t = x.find("+")
    
    return int(x[:t])

In [18]:
df["LS"] = df["LS"].apply(to_base)
df["ST"] = df["ST"].apply(to_base)
df["RS"] = df["RS"].apply(to_base)
df["LW"] = df["LW"].apply(to_base)
df["LF"] = df["LF"].apply(to_base)
df["CF"] = df["CF"].apply(to_base)
df["RF"] = df["RF"].apply(to_base)
df["RW"] = df["RW"].apply(to_base)
df["LAM"] = df["LAM"].apply(to_base)
df["CAM"] = df["CAM"].apply(to_base)
df["RAM"] = df["RAM"].apply(to_base)
df["LM"] = df["LM"].apply(to_base)
df["LCM"] = df["LCM"].apply(to_base)
df["CM"] = df["CM"].apply(to_base)
df["RCM"] = df["RCM"].apply(to_base)
df["RM"] = df["RM"].apply(to_base)
df["LWB"] = df["LWB"].apply(to_base)
df["LDM"] = df["LDM"].apply(to_base)
df["CDM"] = df["CDM"].apply(to_base)
df["RDM"] = df["RDM"].apply(to_base)
df["RWB"] = df["RWB"].apply(to_base)
df["LB"] = df["LB"].apply(to_base)
df["LCB"] = df["LCB"].apply(to_base)
df["CB"] = df["CB"].apply(to_base)
df["RCB"] = df["RCB"].apply(to_base)
df["RB"] = df["RB"].apply(to_base)

df.head()

,Age,Nationality,Overall,Potential,Club,Value,Wage,Preferred Foot,International Reputation,Weak Foot,Skill Moves,Work Rate,Position,Contract Valid Until,Height,Weight,LS,ST,RS,LW,LF,CF,RF,RW,LAM,CAM,RAM,LM,LCM,CM,RCM,RM,LWB,LDM,CDM,RDM,RWB,LB,LCB,CB,RCB,RB,Crossing,Finishing,HeadingAccuracy,ShortPassing,Volleys,Dribbling,Curve,FKAccuracy,LongPassing,BallControl,Acceleration,SprintSpeed,Agility,Reactions,Balance,ShotPower,Jumping,Stamina,Strength,LongShots,Aggression,Interceptions,Positioning,Vision,Penalties,Composure,Marking,StandingTackle,SlidingTackle,GKDiving,GKHandling,GKKicking,GKPositioning,GKReflexes,Release Clause
0,31,Argentina,94,94,FC Barcelona,€110.5M,€565K,Left,5.0,4.0,4.0,Medium/ Medium,RF,2021,5'7,159lbs,88,88,88,92,93,93,93,92,93,93,93,91,84,84,84,91,64,61,61,61,64,59,47,47,47,59,84.0,95.0,70.0,90.0,86.0,97.0,93.0,94.0,87.0,96.0,91.0,86.0,91.0,95.0,95.0,85.0,68.0,72.0,59.0,94.0,48.0,22.0,94.0,94.0,75.0,96.0,33.0,28.0,26.0,6.0,11.0,15.0,14.0,8.0,€226.5M
1,33,Portugal,94,94,Juventus,€77M,€405K,Right,5.0,4.0,5.0,High/ Low,ST,2022,6'2,183lbs,91,91,91,89,90,90,90,89,88,88,88,88,81,81,81,88,65,61,61,61,65,61,53,53,53,61,84.0,94.0,89.0,81.0,87.0,88.0,81.0,76.0,77.0,94.0,89.0,91.0,87.0,96.0,70.0,95.0,95.0,88.0,79.0,93.0,63.0,29.0,95.0,82.0,85.0,95.0,28.0,31.0,23.0,7.0,11.0,15.0,14.0,11.0,€127.1M
2,26,Brazil,92,93,Paris Saint-Germain,€118.5M,€290K,Right,5.0,5.0,5.0,High/ Medium,LW,2022,5'9,150lbs,84,84,84,89,89,89,89,89,89,89,89,88,81,81,81,88,65,60,60,60,65,60,47,47,47,60,79.0,87.0,62.0,84.0,84.0,96.0,88.0,87.0,78.0,95.0,94.0,90.0,96.0,94.0,84.0,80.0,61.0,81.0,49.0,82.0,56.0,36.0,89.0,87.0,81.0,94.0,27.0,24.0,33.0,9.0,9.0,15.0,15.0,11.0,€228.1M
4,27,Belgium,91,92,Manchester City,€102M,€355K,Right,4.0,5.0,4.0,High/ High,RCM,2023,5'11,154lbs,82,82,82,87,87,87,87,87,88,88,88,88,87,87,87,88,77,77,77,77,77,73,66,66,66,73,93.0,82.0,55.0,92.0,82.0,86.0,85.0,83.0,91.0,91.0,78.0,76.0,79.0,91.0,77.0,91.0,63.0,90.0,75.0,91.0,76.0,61.0,87.0,94.0,79.0,88.0,68.0,58.0,51.0,15.0,13.0,5.0,10.0,13.0,€196.4M
5,27,Belgium,91,91,Chelsea,€93M,€340K,Right,4.0,4.0,4.0,High/ Medium,LF,2020,5'8,163lbs,83,83,83,89,88,88,88,89,89,89,89,89,82,82,82,89,66,63,63,63,66,60,49,49,49,60,81.0,84.0,61.0,89.0,80.0,95.0,83.0,79.0,83.0,94.0,94.0,88.0,95.0,90.0,94.0,82.0,56.0,83.0,66.0,80.0,54.0,41.0,87.0,89.0,86.0,91.0,34.0,27.0,22.0,11.0,12.0,6.0,8.0,8.0,€172.1M


Recordant el que s'ha fet en el penúltim pas d'eliminació de NaNs, ens queda assignar els valors de posició dels jugadors porters la mitja de la resta dels jugadors de cada columna. Per extreure la mitja d'una columna, usarem .mean() que ens ofereix la llibreria de pandas. Aquesta funció s'haurà de posar axis = 0 d'argument perquè esculli la mitjana sobre els valors de la columna. 

In [19]:
gk["LS"] = df["LS"].mean(axis = 0)
gk["ST"] = df["ST"].mean(axis = 0)
gk["RS"] = df["RS"].mean(axis = 0)
gk["LW"] = df["LW"].mean(axis = 0)
gk["LF"] = df["LF"].mean(axis = 0)
gk["CF"] = df["CF"].mean(axis = 0)
gk["RF"] = df["RF"].mean(axis = 0)
gk["RW"] = df["RW"].mean(axis = 0)
gk["LAM"] = df["LAM"].mean(axis = 0)
gk["CAM"] = df["CAM"].mean(axis = 0)
gk["RAM"] = df["RAM"].mean(axis = 0)
gk["LM"] = df["LM"].mean(axis = 0)
gk["LCM"] = df["LCM"].mean(axis = 0)
gk["CM"] = df["CM"].mean(axis = 0)
gk["RCM"] = df["RCM"].mean(axis = 0)
gk["RM"] = df["RM"].mean(axis = 0)
gk["LWB"] = df["LWB"].mean(axis = 0)
gk["LDM"] = df["LDM"].mean(axis = 0)
gk["CDM"] = df["CDM"].mean(axis = 0)
gk["RDM"] = df["RDM"].mean(axis = 0)
gk["RWB"] = df["RWB"].mean(axis = 0)
gk["LB"] = df["LB"].mean(axis = 0)
gk["LCB"] = df["LCB"].mean(axis = 0)
gk["CB"] = df["CB"].mean(axis = 0)
gk["RCB"] = df["RCB"].mean(axis = 0)
gk["RB"] = df["RB"].mean(axis = 0)

gk.head()

,Age,Nationality,Overall,Potential,Club,Value,Wage,Preferred Foot,International Reputation,Weak Foot,Skill Moves,Work Rate,Position,Contract Valid Until,Height,Weight,LS,ST,RS,LW,LF,CF,RF,RW,LAM,CAM,RAM,LM,LCM,CM,RCM,RM,LWB,LDM,CDM,RDM,RWB,LB,LCB,CB,RCB,RB,Crossing,Finishing,HeadingAccuracy,ShortPassing,Volleys,Dribbling,Curve,FKAccuracy,LongPassing,BallControl,Acceleration,SprintSpeed,Agility,Reactions,Balance,ShotPower,Jumping,Stamina,Strength,LongShots,Aggression,Interceptions,Positioning,Vision,Penalties,Composure,Marking,StandingTackle,SlidingTackle,GKDiving,GKHandling,GKKicking,GKPositioning,GKReflexes,Release Clause
3,27,Spain,91,93,Manchester United,€72M,€260K,Right,4.0,3.0,1.0,Medium/ Medium,GK,2020,6'4,168lbs,57.821926,57.821926,57.821926,59.046088,58.72975,58.72975,58.72975,59.046088,58.994474,58.994474,58.994474,59.739734,58.223848,58.223848,58.223848,59.739734,57.49617,56.810938,56.810938,56.810938,57.49617,56.785508,55.593307,55.593307,55.593307,56.785508,17.0,13.0,21.0,50.0,13.0,18.0,21.0,19.0,51.0,42.0,57.0,58.0,60.0,90.0,43.0,31.0,67.0,43.0,64.0,12.0,38.0,30.0,12.0,68.0,40.0,68.0,15.0,21.0,13.0,90.0,85.0,87.0,88.0,94.0,€138.6M
9,25,Slovenia,90,93,Atlético Madrid,€68M,€94K,Right,3.0,3.0,1.0,Medium/ Medium,GK,2021,6'2,192lbs,57.821926,57.821926,57.821926,59.046088,58.72975,58.72975,58.72975,59.046088,58.994474,58.994474,58.994474,59.739734,58.223848,58.223848,58.223848,59.739734,57.49617,56.810938,56.810938,56.810938,57.49617,56.785508,55.593307,55.593307,55.593307,56.785508,13.0,11.0,15.0,29.0,13.0,12.0,13.0,14.0,26.0,16.0,43.0,60.0,67.0,86.0,49.0,22.0,76.0,41.0,78.0,12.0,34.0,19.0,11.0,70.0,11.0,70.0,27.0,12.0,18.0,86.0,92.0,78.0,88.0,89.0,€144.5M
18,26,Germany,89,92,FC Barcelona,€58M,€240K,Right,3.0,4.0,1.0,Medium/ Medium,GK,2022,6'2,187lbs,57.821926,57.821926,57.821926,59.046088,58.72975,58.72975,58.72975,59.046088,58.994474,58.994474,58.994474,59.739734,58.223848,58.223848,58.223848,59.739734,57.49617,56.810938,56.810938,56.810938,57.49617,56.785508,55.593307,55.593307,55.593307,56.785508,15.0,14.0,11.0,36.0,14.0,17.0,18.0,12.0,42.0,18.0,38.0,50.0,37.0,85.0,43.0,22.0,79.0,35.0,79.0,10.0,43.0,22.0,11.0,69.0,25.0,69.0,25.0,13.0,10.0,87.0,85.0,88.0,85.0,90.0,€123.3M
19,26,Belgium,89,90,Real Madrid,€53.5M,€240K,Left,4.0,2.0,1.0,Medium/ Medium,GK,2024,6'6,212lbs,57.821926,57.821926,57.821926,59.046088,58.72975,58.72975,58.72975,59.046088,58.994474,58.994474,58.994474,59.739734,58.223848,58.223848,58.223848,59.739734,57.49617,56.810938,56.810938,56.810938,57.49617,56.785508,55.593307,55.593307,55.593307,56.785508,14.0,14.0,13.0,33.0,12.0,13.0,19.0,20.0,35.0,23.0,46.0,52.0,61.0,84.0,45.0,36.0,68.0,38.0,70.0,17.0,23.0,15.0,13.0,44.0,27.0,66.0,20.0,18.0,16.0,85.0,91.0,72.0,86.0,88.0,€113.7M
22,32,Germany,89,89,FC Bayern München,€38M,€130K,Right,5.0,4.0,1.0,Medium/ Medium,GK,2021,6'4,203lbs,57.821926,57.821926,57.821926,59.046088,58.72975,58.72975,58.72975,59.046088,58.994474,58.994474,58.994474,59.739734,58.223848,58.223848,58.223848,59.739734,57.49617,56.810938,56.810938,56.810938,57.49617,56.785508,55.593307,55.593307,55.593307,56.785508,15.0,13.0,25.0,55.0,11.0,30.0,14.0,11.0,59.0,48.0,54.0,60.0,51.0,84.0,35.0,25.0,77.0,43.0,80.0,16.0,29.0,30.0,12.0,70.0,47.0,70.0,17.0,10.0,11.0,90.0,86.0,91.0,87.0,87.0,€62.7M


Finalment procedirem a juntar aquest dataset auxiliar amb el original: 

In [20]:
aux = [df,gk]
df = pd.concat(aux)
df

,Age,Nationality,Overall,Potential,Club,Value,Wage,Preferred Foot,International Reputation,Weak Foot,Skill Moves,Work Rate,Position,Contract Valid Until,Height,Weight,LS,ST,RS,LW,LF,CF,RF,RW,LAM,CAM,RAM,LM,LCM,CM,RCM,RM,LWB,LDM,CDM,RDM,RWB,LB,LCB,CB,RCB,RB,Crossing,Finishing,HeadingAccuracy,ShortPassing,Volleys,Dribbling,Curve,FKAccuracy,LongPassing,BallControl,Acceleration,SprintSpeed,Agility,Reactions,Balance,ShotPower,Jumping,Stamina,Strength,LongShots,Aggression,Interceptions,Positioning,Vision,Penalties,Composure,Marking,StandingTackle,SlidingTackle,GKDiving,GKHandling,GKKicking,GKPositioning,GKReflexes,Release Clause
0,31,Argentina,94,94,FC Barcelona,€110.5M,€565K,Left,5.0,4.0,4.0,Medium/ Medium,RF,2021,5'7,159lbs,88.000000,88.000000,88.000000,92.000000,93.00000,93.00000,93.00000,92.000000,93.000000,93.000000,93.000000,91.000000,84.000000,84.000000,84.000000,91.000000,64.00000,61.000000,61.000000,61.000000,64.00000,59.000000,47.000000,47.000000,47.000000,59.000000,84.0,95.0,70.0,90.0,86.0,97.0,93.0,94.0,87.0,96.0,91.0,86.0,91.0,95.0,95.0,85.0,68.0,72.0,59.0,94.0,48.0,22.0,94.0,94.0,75.0,96.0,33.0,28.0,26.0,6.0,11.0,15.0,14.0,8.0,€226.5M
1,33,Portugal,94,94,Juventus,€77M,€405K,Right,5.0,4.0,5.0,High/ Low,ST,2022,6'2,183lbs,91.000000,91.000000,91.000000,89.000000,90.00000,90.00000,90.00000,89.000000,88.000000,88.000000,88.000000,88.000000,81.000000,81.000000,81.000000,88.000000,65.00000,61.000000,61.000000,61.000000,65.00000,61.000000,53.000000,53.000000,53.000000,61.000000,84.0,94.0,89.0,81.0,87.0,88.0,81.0,76.0,77.0,94.0,89.0,91.0,87.0,96.0,70.0,95.0,95.0,88.0,79.0,93.0,63.0,29.0,95.0,82.0,85.0,95.0,28.0,31.0,23.0,7.0,11.0,15.0,14.0,11.0,€127.1M
2,26,Brazil,92,93,Paris Saint-Germain,€118.5M,€290K,Right,5.0,5.0,5.0,High/ Medium,LW,2022,5'9,150lbs,84.000000,84.000000,84.000000,89.000000,89.00000,89.00000,89.00000,89.000000,89.000000,89.000000,89.000000,88.000000,81.000000,81.000000,81.000000,88.000000,65.00000,60.000000,60.000000,60.000000,65.00000,60.000000,47.000000,47.000000,47.000000,60.000000,79.0,87.0,62.0,84.0,84.0,96.0,88.0,87.0,78.0,95.0,94.0,90.0,96.0,94.0,84.0,80.0,61.0,81.0,49.0,82.0,56.0,36.0,89.0,87.0,81.0,94.0,27.0,24.0,33.0,9.0,9.0,15.0,15.0,11.0,€228.1M
4,27,Belgium,91,92,Manchester City,€102M,€355K,Right,4.0,5.0,4.0,High/ High,RCM,2023,5'11,154lbs,82.000000,82.000000,82.000000,87.000000,87.00000,87.00000,87.00000,87.000000,88.000000,88.000000,88.000000,88.000000,87.000000,87.000000,87.000000,88.000000,77.00000,77.000000,77.000000,77.000000,77.00000,73.000000,66.000000,66.000000,66.000000,73.000000,93.0,82.0,55.0,92.0,82.0,86.0,85.0,83.0,91.0,91.0,78.0,76.0,79.0,91.0,77.0,91.0,63.0,90.0,75.0,91.0,76.0,61.0,87.0,94.0,79.0,88.0,68.0,58.0,51.0,15.0,13.0,5.0,10.0,13.0,€196.4M
5,27,Belgium,91,91,Chelsea,€93M,€340K,Right,4.0,4.0,4.0,High/ Medium,LF,2020,5'8,163lbs,83.000000,83.000000,83.000000,89.000000,88.00000,88.00000,88.00000,89.000000,89.000000,89.000000,89.000000,89.000000,82.000000,82.000000,82.000000,89.000000,66.00000,63.000000,63.000000,63.000000,66.00000,60.000000,49.000000,49.000000,49.000000,60.000000,81.0,84.0,61.0,89.0,80.0,95.0,83.0,79.0,83.0,94.0,94.0,88.0,95.0,90.0,94.0,82.0,56.0,83.0,66.0,80.0,54.0,41.0,87.0,89.0,86.0,91.0,34.0,27.0,22.0,11.0,12.0,6.0,8.0,8.0,€172.1M
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18178,18,Sweden,48,65,Dalkurd FF,€50K,€1K,Right,1.0,2.0,1.0,Medium/ Medium,GK,2018,6'0,176lbs,57.821926,57.821926,57.821926,59.046088,58.72975,58.72975,58.72975,59.046088,58.994474,58.994474,58.994474,59.739734,58.223848,58.223848,58.223848,59.739734,57.49617,56.810938,56.810938,56.810938,57.49617,56.785508,55.593307,55.593307,55.593307,56.785508,10.0,7.0,11.0,22.0,6.0,8.0,10.0,8.0,20.0,12.0,24.0,28.0,33.0,40.0,29.0,15.0,48.

Per transformar els valor monetaris com "Value", "Wage" i "Release Clause" a un nombre enter farem el següent:

In [21]:
def value_to_float(x):
    """
    From K and M to float.
    
    """
    x = x.replace('€', '')
    ret_val = 0.0
    
    if type(x) == float or type(x) == int:
        ret_val = x
    if 'K' in x:
        if len(x) > 1:
            ret_val = float(x.replace('K', ''))
        ret_val = ret_val *1000
    if 'M' in x:
        if len(x) > 1:
            ret_val = float(x.replace('M', ''))
        ret_val = ret_val * 1000000.0
    return ret_val

In [22]:
df["Value"] = df["Value"].apply(value_to_float)
df["Wage"] = df["Wage"].apply(value_to_float)
df["Release Clause"] = df["Release Clause"].apply(value_to_float)
df.head()


,Age,Nationality,Overall,Potential,Club,Value,Wage,Preferred Foot,International Reputation,Weak Foot,Skill Moves,Work Rate,Position,Contract Valid Until,Height,Weight,LS,ST,RS,LW,LF,CF,RF,RW,LAM,CAM,RAM,LM,LCM,CM,RCM,RM,LWB,LDM,CDM,RDM,RWB,LB,LCB,CB,RCB,RB,Crossing,Finishing,HeadingAccuracy,ShortPassing,Volleys,Dribbling,Curve,FKAccuracy,LongPassing,BallControl,Acceleration,SprintSpeed,Agility,Reactions,Balance,ShotPower,Jumping,Stamina,Strength,LongShots,Aggression,Interceptions,Positioning,Vision,Penalties,Composure,Marking,StandingTackle,SlidingTackle,GKDiving,GKHandling,GKKicking,GKPositioning,GKReflexes,Release Clause
0,31,Argentina,94,94,FC Barcelona,110500000.0,565000.0,Left,5.0,4.0,4.0,Medium/ Medium,RF,2021,5'7,159lbs,88.0,88.0,88.0,92.0,93.0,93.0,93.0,92.0,93.0,93.0,93.0,91.0,84.0,84.0,84.0,91.0,64.0,61.0,61.0,61.0,64.0,59.0,47.0,47.0,47.0,59.0,84.0,95.0,70.0,90.0,86.0,97.0,93.0,94.0,87.0,96.0,91.0,86.0,91.0,95.0,95.0,85.0,68.0,72.0,59.0,94.0,48.0,22.0,94.0,94.0,75.0,96.0,33.0,28.0,26.0,6.0,11.0,15.0,14.0,8.0,226500000.0
1,33,Portugal,94,94,Juventus,77000000.0,405000.0,Right,5.0,4.0,5.0,High/ Low,ST,2022,6'2,183lbs,91.0,91.0,91.0,89.0,90.0,90.0,90.0,89.0,88.0,88.0,88.0,88.0,81.0,81.0,81.0,88.0,65.0,61.0,61.0,61.0,65.0,61.0,53.0,53.0,53.0,61.0,84.0,94.0,89.0,81.0,87.0,88.0,81.0,76.0,77.0,94.0,89.0,91.0,87.0,96.0,70.0,95.0,95.0,88.0,79.0,93.0,63.0,29.0,95.0,82.0,85.0,95.0,28.0,31.0,23.0,7.0,11.0,15.0,14.0,11.0,127100000.0
2,26,Brazil,92,93,Paris Saint-Germain,118500000.0,290000.0,Right,5.0,5.0,5.0,High/ Medium,LW,2022,5'9,150lbs,84.0,84.0,84.0,89.0,89.0,89.0,89.0,89.0,89.0,89.0,89.0,88.0,81.0,81.0,81.0,88.0,65.0,60.0,60.0,60.0,65.0,60.0,47.0,47.0,47.0,60.0,79.0,87.0,62.0,84.0,84.0,96.0,88.0,87.0,78.0,95.0,94.0,90.0,96.0,94.0,84.0,80.0,61.0,81.0,49.0,82.0,56.0,36.0,89.0,87.0,81.0,94.0,27.0,24.0,33.0,9.0,9.0,15.0,15.0,11.0,228100000.0
4,27,Belgium,91,92,Manchester City,102000000.0,355000.0,Right,4.0,5.0,4.0,High/ High,RCM,2023,5'11,154lbs,82.0,82.0,82.0,87.0,87.0,87.0,87.0,87.0,88.0,88.0,88.0,88.0,87.0,87.0,87.0,88.0,77.0,77.0,77.0,77.0,77.0,73.0,66.0,66.0,66.0,73.0,93.0,82.0,55.0,92.0,82.0,86.0,85.0,83.0,91.0,91.0,78.0,76.0,79.0,91.0,77.0,91.0,63.0,90.0,75.0,91.0,76.0,61.0,87.0,94.0,79.0,88.0,68.0,58.0,51.0,15.0,13.0,5.0,10.0,13.0,196400000.0
5,27,Belgium,91,91,Chelsea,93000000.0,340000.0,Right,4.0,4.0,4.0,High/ Medium,LF,2020,5'8,163lbs,83.0,83.0,83.0,89.0,88.0,88.0,88.0,89.0,89.0,89.0,89.0,89.0,82.0,82.0,82.0,89.0,66.0,63.0,63.0,63.0,66.0,60.0,49.0,49.0,49.0,60.0,81.0,84.0,61.0,89.0,80.0,95.0,83.0,79.0,83.0,94.0,94.0,88.0,95.0,90.0,94.0,82.0,56.0,83.0,66.0,80.0,54.0,41.0,87.0,89.0,86.0,91.0,34.0,27.0,22.0,11.0,12.0,6.0,8.0,8.0,172100000.0


In [23]:
def date_to_year(x):
    """
    From date (DD/MM/YYYY) to YYYY.
    
    """
    # Si la es troba en format YYYY directament es fa el return
    if(len(x) == 4):
        return int(x)
    year = x[-4:]
    return int(year)

In [24]:
df["Contract Valid Until"] = df["Contract Valid Until"].apply(date_to_year)
df['Contract Valid Until'].unique()

array([2021, 2022, 2023, 2020, 2019, 2024, 2026, 2018, 2025], dtype=int64)

L'altura dels jugadors "Height", està expresada en peus (X'Y), per tant es passarà a centímetres: 

In [25]:
def to_cm(x):
    """
    From USA height format to SI.
    
    """
    height = x.split('\'')
    feet = int(height[0])
    inches = int(height[1])
    cm = feet*30.48 + inches*2.54
    return cm

In [26]:
df["Height"] = df["Height"].apply(to_cm)
df.head()

,Age,Nationality,Overall,Potential,Club,Value,Wage,Preferred Foot,International Reputation,Weak Foot,Skill Moves,Work Rate,Position,Contract Valid Until,Height,Weight,LS,ST,RS,LW,LF,CF,RF,RW,LAM,CAM,RAM,LM,LCM,CM,RCM,RM,LWB,LDM,CDM,RDM,RWB,LB,LCB,CB,RCB,RB,Crossing,Finishing,HeadingAccuracy,ShortPassing,Volleys,Dribbling,Curve,FKAccuracy,LongPassing,BallControl,Acceleration,SprintSpeed,Agility,Reactions,Balance,ShotPower,Jumping,Stamina,Strength,LongShots,Aggression,Interceptions,Positioning,Vision,Penalties,Composure,Marking,StandingTackle,SlidingTackle,GKDiving,GKHandling,GKKicking,GKPositioning,GKReflexes,Release Clause
0,31,Argentina,94,94,FC Barcelona,110500000.0,565000.0,Left,5.0,4.0,4.0,Medium/ Medium,RF,2021,170.18,159lbs,88.0,88.0,88.0,92.0,93.0,93.0,93.0,92.0,93.0,93.0,93.0,91.0,84.0,84.0,84.0,91.0,64.0,61.0,61.0,61.0,64.0,59.0,47.0,47.0,47.0,59.0,84.0,95.0,70.0,90.0,86.0,97.0,93.0,94.0,87.0,96.0,91.0,86.0,91.0,95.0,95.0,85.0,68.0,72.0,59.0,94.0,48.0,22.0,94.0,94.0,75.0,96.0,33.0,28.0,26.0,6.0,11.0,15.0,14.0,8.0,226500000.0
1,33,Portugal,94,94,Juventus,77000000.0,405000.0,Right,5.0,4.0,5.0,High/ Low,ST,2022,187.96,183lbs,91.0,91.0,91.0,89.0,90.0,90.0,90.0,89.0,88.0,88.0,88.0,88.0,81.0,81.0,81.0,88.0,65.0,61.0,61.0,61.0,65.0,61.0,53.0,53.0,53.0,61.0,84.0,94.0,89.0,81.0,87.0,88.0,81.0,76.0,77.0,94.0,89.0,91.0,87.0,96.0,70.0,95.0,95.0,88.0,79.0,93.0,63.0,29.0,95.0,82.0,85.0,95.0,28.0,31.0,23.0,7.0,11.0,15.0,14.0,11.0,127100000.0
2,26,Brazil,92,93,Paris Saint-Germain,118500000.0,290000.0,Right,5.0,5.0,5.0,High/ Medium,LW,2022,175.26,150lbs,84.0,84.0,84.0,89.0,89.0,89.0,89.0,89.0,89.0,89.0,89.0,88.0,81.0,81.0,81.0,88.0,65.0,60.0,60.0,60.0,65.0,60.0,47.0,47.0,47.0,60.0,79.0,87.0,62.0,84.0,84.0,96.0,88.0,87.0,78.0,95.0,94.0,90.0,96.0,94.0,84.0,80.0,61.0,81.0,49.0,82.0,56.0,36.0,89.0,87.0,81.0,94.0,27.0,24.0,33.0,9.0,9.0,15.0,15.0,11.0,228100000.0
4,27,Belgium,91,92,Manchester City,102000000.0,355000.0,Right,4.0,5.0,4.0,High/ High,RCM,2023,180.34,154lbs,82.0,82.0,82.0,87.0,87.0,87.0,87.0,87.0,88.0,88.0,88.0,88.0,87.0,87.0,87.0,88.0,77.0,77.0,77.0,77.0,77.0,73.0,66.0,66.0,66.0,73.0,93.0,82.0,55.0,92.0,82.0,86.0,85.0,83.0,91.0,91.0,78.0,76.0,79.0,91.0,77.0,91.0,63.0,90.0,75.0,91.0,76.0,61.0,87.0,94.0,79.0,88.0,68.0,58.0,51.0,15.0,13.0,5.0,10.0,13.0,196400000.0
5,27,Belgium,91,91,Chelsea,93000000.0,340000.0,Right,4.0,4.0,4.0,High/ Medium,LF,2020,172.72,163lbs,83.0,83.0,83.0,89.0,88.0,88.0,88.0,89.0,89.0,89.0,89.0,89.0,82.0,82.0,82.0,89.0,66.0,63.0,63.0,63.0,66.0,60.0,49.0,49.0,49.0,60.0,81.0,84.0,61.0,89.0,80.0,95.0,83.0,79.0,83.0,94.0,94.0,88.0,95.0,90.0,94.0,82.0,56.0,83.0,66.0,80.0,54.0,41.0,87.0,89.0,86.0,91.0,34.0,27.0,22.0,11.0,12.0,6.0,8.0,8.0,172100000.0


Eliminarem les unitats de mesura en el pes dels jugadors "Weight", expresat en lbs

In [27]:
def weight_to_int(x):
    """
    Remove lbs at the end of every "Weight" value
    
    """
    return int(x[:len(x)-3])

In [28]:
df["Weight"] = df["Weight"].apply(weight_to_int)
df.head()

,Age,Nationality,Overall,Potential,Club,Value,Wage,Preferred Foot,International Reputation,Weak Foot,Skill Moves,Work Rate,Position,Contract Valid Until,Height,Weight,LS,ST,RS,LW,LF,CF,RF,RW,LAM,CAM,RAM,LM,LCM,CM,RCM,RM,LWB,LDM,CDM,RDM,RWB,LB,LCB,CB,RCB,RB,Crossing,Finishing,HeadingAccuracy,ShortPassing,Volleys,Dribbling,Curve,FKAccuracy,LongPassing,BallControl,Acceleration,SprintSpeed,Agility,Reactions,Balance,ShotPower,Jumping,Stamina,Strength,LongShots,Aggression,Interceptions,Positioning,Vision,Penalties,Composure,Marking,StandingTackle,SlidingTackle,GKDiving,GKHandling,GKKicking,GKPositioning,GKReflexes,Release Clause
0,31,Argentina,94,94,FC Barcelona,110500000.0,565000.0,Left,5.0,4.0,4.0,Medium/ Medium,RF,2021,170.18,159,88.0,88.0,88.0,92.0,93.0,93.0,93.0,92.0,93.0,93.0,93.0,91.0,84.0,84.0,84.0,91.0,64.0,61.0,61.0,61.0,64.0,59.0,47.0,47.0,47.0,59.0,84.0,95.0,70.0,90.0,86.0,97.0,93.0,94.0,87.0,96.0,91.0,86.0,91.0,95.0,95.0,85.0,68.0,72.0,59.0,94.0,48.0,22.0,94.0,94.0,75.0,96.0,33.0,28.0,26.0,6.0,11.0,15.0,14.0,8.0,226500000.0
1,33,Portugal,94,94,Juventus,77000000.0,405000.0,Right,5.0,4.0,5.0,High/ Low,ST,2022,187.96,183,91.0,91.0,91.0,89.0,90.0,90.0,90.0,89.0,88.0,88.0,88.0,88.0,81.0,81.0,81.0,88.0,65.0,61.0,61.0,61.0,65.0,61.0,53.0,53.0,53.0,61.0,84.0,94.0,89.0,81.0,87.0,88.0,81.0,76.0,77.0,94.0,89.0,91.0,87.0,96.0,70.0,95.0,95.0,88.0,79.0,93.0,63.0,29.0,95.0,82.0,85.0,95.0,28.0,31.0,23.0,7.0,11.0,15.0,14.0,11.0,127100000.0
2,26,Brazil,92,93,Paris Saint-Germain,118500000.0,290000.0,Right,5.0,5.0,5.0,High/ Medium,LW,2022,175.26,150,84.0,84.0,84.0,89.0,89.0,89.0,89.0,89.0,89.0,89.0,89.0,88.0,81.0,81.0,81.0,88.0,65.0,60.0,60.0,60.0,65.0,60.0,47.0,47.0,47.0,60.0,79.0,87.0,62.0,84.0,84.0,96.0,88.0,87.0,78.0,95.0,94.0,90.0,96.0,94.0,84.0,80.0,61.0,81.0,49.0,82.0,56.0,36.0,89.0,87.0,81.0,94.0,27.0,24.0,33.0,9.0,9.0,15.0,15.0,11.0,228100000.0
4,27,Belgium,91,92,Manchester City,102000000.0,355000.0,Right,4.0,5.0,4.0,High/ High,RCM,2023,180.34,154,82.0,82.0,82.0,87.0,87.0,87.0,87.0,87.0,88.0,88.0,88.0,88.0,87.0,87.0,87.0,88.0,77.0,77.0,77.0,77.0,77.0,73.0,66.0,66.0,66.0,73.0,93.0,82.0,55.0,92.0,82.0,86.0,85.0,83.0,91.0,91.0,78.0,76.0,79.0,91.0,77.0,91.0,63.0,90.0,75.0,91.0,76.0,61.0,87.0,94.0,79.0,88.0,68.0,58.0,51.0,15.0,13.0,5.0,10.0,13.0,196400000.0
5,27,Belgium,91,91,Chelsea,93000000.0,340000.0,Right,4.0,4.0,4.0,High/ Medium,LF,2020,172.72,163,83.0,83.0,83.0,89.0,88.0,88.0,88.0,89.0,89.0,89.0,89.0,89.0,82.0,82.0,82.0,89.0,66.0,63.0,63.0,63.0,66.0,60.0,49.0,49.0,49.0,60.0,81.0,84.0,61.0,89.0,80.0,95.0,83.0,79.0,83.0,94.0,94.0,88.0,95.0,90.0,94.0,82.0,56.0,83.0,66.0,80.0,54.0,41.0,87.0,89.0,86.0,91.0,34.0,27.0,22.0,11.0,12.0,6.0,8.0,8.0,172100000.0


### Conversió de Strings

L'algoritme de Machine Learning no sap tractar els atributs en format String, per tant aquests s'hauran de transformar a algun format que l'algoritme ho pugui entendre, per això usarem la funció get_dummies proporcionada per la llibreria ``panda``.

Per usar el Club al que pertany un jugador per determinar el seu preu de mercat, al trobar-se en format String s'haurà de transformar:

In [29]:
clb = df.pop("Club")

df = pd.concat([df.reset_index(drop=True), pd.get_dummies(clb, prefix='clb').reset_index(drop=True)], axis=1, sort=False)
df

,Age,Nationality,Overall,Potential,Value,Wage,Preferred Foot,International Reputation,Weak Foot,Skill Moves,Work Rate,Position,Contract Valid Until,Height,Weight,LS,ST,RS,LW,LF,CF,RF,RW,LAM,CAM,RAM,LM,LCM,CM,RCM,RM,LWB,LDM,CDM,RDM,RWB,LB,LCB,CB,RCB,RB,Crossing,Finishing,HeadingAccuracy,ShortPassing,Volleys,Dribbling,Curve,FKAccuracy,LongPassing,BallControl,Acceleration,SprintSpeed,Agility,Reactions,Balance,ShotPower,Jumping,Stamina,Strength,LongShots,Aggression,Interceptions,Positioning,Vision,Penalties,Composure,Marking,StandingTackle,SlidingTackle,GKDiving,GKHandling,GKKicking,GKPositioning,GKReflexes,Release Clause,clb_ SSV Jahn Regensburg,clb_1. FC Heidenheim 1846,clb_1. FC Kaiserslautern,clb_1. FC Köln,clb_1. FC Magdeburg,clb_1. FC Nürnberg,clb_1. FC Union Berlin,clb_1. FSV Mainz 05,clb_AC Ajaccio,clb_AC Horsens,clb_AD Alcorcón,clb_ADO Den Haag,clb_AEK Athens,clb_AFC Wimbledon,clb_AIK,clb_AJ Auxerre,clb_AS Béziers,clb_AS Monaco,clb_AS Nancy Lorraine,clb_AS Saint-Étienne,clb_AZ Alkmaar,clb_Aalborg BK,clb_Aarhus GF,clb_Aberdeen,clb_Accrington Stanley,clb_Adelaide United,clb_Ajax,clb_Akhisar Belediyespor,clb_Al Ahli,clb_Al Batin,clb_Al Faisaly,clb_Al Fateh,clb_Al Fayha,clb_Al Hazem,clb_Al Hilal,clb_Al Ittihad,clb_Al Nassr,clb_Al Qadisiyah,clb_Al Raed,clb_Al Shabab,clb_Al Taawoun,clb_Al Wehda,clb_Alanyaspor,clb_Albacete BP,clb_Alianza Petrolera,clb_Amiens SC,clb_América FC (Minas Gerais),clb_América de Cali,clb_Angers SCO,clb_Antalyaspor,clb_Argentinos Juniors,clb_Arka Gdynia,clb_Arsenal,clb_Ascoli,clb_Aston Villa,clb_Atalanta,clb_Athletic Club de Bilbao,clb_Atiker Konyaspor,clb_Atlanta United,clb_Atlético Bucaramanga,clb_Atlético Huila,clb_Atlético Madrid,clb_Atlético Mineiro,clb_Atlético Nacional,clb_Atlético Paranaense,clb_Atlético Tucumán,clb_Audax Italiano,clb_BB Erzurumspor,clb_BK Häcken,clb_BSC Young Boys,clb_Bahia,clb_Barnsley,clb_Bayer 04 Leverkusen,clb_Beijing Renhe FC,clb_Beijing Sinobo Guoan FC,clb_Belgrano de Córdoba,clb_Benevento,clb_Beşiktaş JK,clb_Birmingham City,clb_Blackburn Rovers,clb_Blackpool,clb_Boavista FC,clb_Boca Juniors,clb_Bohemian FC,clb_Bologna,clb_Bolton Wanderers,clb_Borussia Dortmund,clb_Borussia Mönchengladbach,clb_Botafogo,clb_Bournemouth,clb_Boyacá Chicó FC,clb_Bradford City,clb_Bray Wanderers,clb_Brentford,clb_Brescia,clb_Brighton & Hove Albion,clb_Brisbane Roar,clb_Bristol City,clb_Bristol Rovers,clb_Brøndby IF,clb_Burnley,clb_Bursaspor,clb_Burton Albion,clb_Bury,clb_CA Osasuna,clb_CD Antofagasta,clb_CD Aves,clb_CD Everton de Viña del Mar,clb_CD Feirense,clb_CD Huachipato,clb_CD Leganés,clb_CD Lugo,clb_CD Nacional,clb_CD Numancia,clb_CD O'Higgins,clb_CD Palestino,clb_CD Tenerife,clb_CD Tondela,clb_CD Universidad de Concepción,clb_CF Rayo Majadahonda,clb_CF Reus Deportiu,clb_Cagliari,clb_Cambridge United,clb_Cardiff City,clb_Carlisle United,clb_Carpi,clb_Ceará Sporting Club,clb_Celtic,clb_Central Coast Mariners,clb_Cerezo Osaka,clb_Chamois Niortais Football Club,clb_Changchun Yatai FC,clb_Chapecoense,clb_Charlton Athletic,clb_Chelsea,clb_Cheltenham Town,clb_Chicago Fire,clb_Chievo Verona,clb_Chongqing Dangdai Lifan FC SWM Team,clb_Cittadella,clb_Clermont Foot 63,clb_Club América,clb_Club Atlas,clb_Club Atlético Aldosivi,clb_Club Atlético Banfield,clb_Club Atlético Colón,clb_Club Atlético Huracán,clb_Club Atlético Lanús,clb_Club Atlético Talleres,clb_Club Atlético Tigre,clb_Club Brugge KV,clb_Club Deportes Temuco,clb_Club León,clb_Club Necaxa,clb_Club Tijuana,clb_Clube Sport Marítimo,clb_Colchester United,clb_Colo-Colo,clb_Colorado Rapids,clb_Columbus Crew SC,clb_Cork City,clb_Cosenza,clb_Coventry City,clb_Cracovia,clb_Crawley Town,clb_Crewe Alexandra,clb_Crotone,clb_Cruz Azul,clb_Cruzeiro,clb_Crystal Palace,clb_Curicó Unido,clb_Cádiz CF,clb_Córdoba CF,clb_DC United,clb_DSC Arminia Bielefeld,clb_Daegu FC,clb_Dalian YiFang FC,clb_Dalkurd FF,clb_De Graafschap,clb_Defensa y Justicia,clb_Deportes Iquique,clb_Deportes Tolima,clb_Deportivo Alavés,clb_Deportivo Cali,clb_Deportivo Pasto,clb_Deportiv

Usarem la mateixa tècnica per transformar els atributs "Nationality", "Work Rate", "Position", i "Preferred Foot", ja que tenen la mateixa problemàtica.

In [30]:
nat = df.pop("Nationality")
df = pd.concat([df.reset_index(drop=True), pd.get_dummies(nat, prefix='nat').reset_index(drop=True)], axis=1, sort=False)

wr = df.pop("Work Rate")
df = pd.concat([df.reset_index(drop=True), pd.get_dummies(wr, prefix='wr').reset_index(drop=True)], axis=1, sort=False)

pos  = df.pop("Position")
df = pd.concat([df.reset_index(drop=True), pd.get_dummies(pos, prefix='pos').reset_index(drop=True)], axis=1, sort=False)

pf  = df.pop("Preferred Foot")
df = pd.concat([df.reset_index(drop=True), pd.get_dummies(pf, prefix='pf').reset_index(drop=True)], axis=1, sort=False)

df.head()

,Age,Overall,Potential,Value,Wage,International Reputation,Weak Foot,Skill Moves,Contract Valid Until,Height,Weight,LS,ST,RS,LW,LF,CF,RF,RW,LAM,CAM,RAM,LM,LCM,CM,RCM,RM,LWB,LDM,CDM,RDM,RWB,LB,LCB,CB,RCB,RB,Crossing,Finishing,HeadingAccuracy,ShortPassing,Volleys,Dribbling,Curve,FKAccuracy,LongPassing,BallControl,Acceleration,SprintSpeed,Agility,Reactions,Balance,ShotPower,Jumping,Stamina,Strength,LongShots,Aggression,Interceptions,Positioning,Vision,Penalties,Composure,Marking,StandingTackle,SlidingTackle,GKDiving,GKHandling,GKKicking,GKPositioning,GKReflexes,Release Clause,clb_ SSV Jahn Regensburg,clb_1. FC Heidenheim 1846,clb_1. FC Kaiserslautern,clb_1. FC Köln,clb_1. FC Magdeburg,clb_1. FC Nürnberg,clb_1. FC Union Berlin,clb_1. FSV Mainz 05,clb_AC Ajaccio,clb_AC Horsens,clb_AD Alcorcón,clb_ADO Den Haag,clb_AEK Athens,clb_AFC Wimbledon,clb_AIK,clb_AJ Auxerre,clb_AS Béziers,clb_AS Monaco,clb_AS Nancy Lorraine,clb_AS Saint-Étienne,clb_AZ Alkmaar,clb_Aalborg BK,clb_Aarhus GF,clb_Aberdeen,clb_Accrington Stanley,clb_Adelaide United,clb_Ajax,clb_Akhisar Belediyespor,clb_Al Ahli,clb_Al Batin,clb_Al Faisaly,clb_Al Fateh,clb_Al Fayha,clb_Al Hazem,clb_Al Hilal,clb_Al Ittihad,clb_Al Nassr,clb_Al Qadisiyah,clb_Al Raed,clb_Al Shabab,clb_Al Taawoun,clb_Al Wehda,clb_Alanyaspor,clb_Albacete BP,clb_Alianza Petrolera,clb_Amiens SC,clb_América FC (Minas Gerais),clb_América de Cali,clb_Angers SCO,clb_Antalyaspor,clb_Argentinos Juniors,clb_Arka Gdynia,clb_Arsenal,clb_Ascoli,clb_Aston Villa,clb_Atalanta,clb_Athletic Club de Bilbao,clb_Atiker Konyaspor,clb_Atlanta United,clb_Atlético Bucaramanga,clb_Atlético Huila,clb_Atlético Madrid,clb_Atlético Mineiro,clb_Atlético Nacional,clb_Atlético Paranaense,clb_Atlético Tucumán,clb_Audax Italiano,clb_BB Erzurumspor,clb_BK Häcken,clb_BSC Young Boys,clb_Bahia,clb_Barnsley,clb_Bayer 04 Leverkusen,clb_Beijing Renhe FC,clb_Beijing Sinobo Guoan FC,clb_Belgrano de Córdoba,clb_Benevento,clb_Beşiktaş JK,clb_Birmingham City,clb_Blackburn Rovers,clb_Blackpool,clb_Boavista FC,clb_Boca Juniors,clb_Bohemian FC,clb_Bologna,clb_Bolton Wanderers,clb_Borussia Dortmund,clb_Borussia Mönchengladbach,clb_Botafogo,clb_Bournemouth,clb_Boyacá Chicó FC,clb_Bradford City,clb_Bray Wanderers,clb_Brentford,clb_Brescia,clb_Brighton & Hove Albion,clb_Brisbane Roar,clb_Bristol City,clb_Bristol Rovers,clb_Brøndby IF,clb_Burnley,clb_Bursaspor,clb_Burton Albion,clb_Bury,clb_CA Osasuna,clb_CD Antofagasta,clb_CD Aves,clb_CD Everton de Viña del Mar,clb_CD Feirense,clb_CD Huachipato,clb_CD Leganés,clb_CD Lugo,clb_CD Nacional,clb_CD Numancia,clb_CD O'Higgins,clb_CD Palestino,clb_CD Tenerife,clb_CD Tondela,clb_CD Universidad de Concepción,clb_CF Rayo Majadahonda,clb_CF Reus Deportiu,clb_Cagliari,clb_Cambridge United,clb_Cardiff City,clb_Carlisle United,clb_Carpi,clb_Ceará Sporting Club,clb_Celtic,clb_Central Coast Mariners,clb_Cerezo Osaka,clb_Chamois Niortais Football Club,clb_Changchun Yatai FC,clb_Chapecoense,clb_Charlton Athletic,clb_Chelsea,clb_Cheltenham Town,clb_Chicago Fire,clb_Chievo Verona,clb_Chongqing Dangdai Lifan FC SWM Team,clb_Cittadella,clb_Clermont Foot 63,clb_Club América,clb_Club Atlas,clb_Club Atlético Aldosivi,clb_Club Atlético Banfield,clb_Club Atlético Colón,clb_Club Atlético Huracán,clb_Club Atlético Lanús,clb_Club Atlético Talleres,clb_Club Atlético Tigre,clb_Club Brugge KV,clb_Club Deportes Temuco,clb_Club León,clb_Club Necaxa,clb_Club Tijuana,clb_Clube Sport Marítimo,clb_Colchester United,clb_Colo-Colo,clb_Colorado Rapids,clb_Columbus Crew SC,clb_Cork City,clb_Cosenza,clb_Coventry City,clb_Cracovia,clb_Crawley Town,clb_Crewe Alexandra,clb_Crotone,clb_Cruz Azul,clb_Cruzeiro,clb_Crystal Palace,clb_Curicó Unido,clb_Cádiz CF,clb_Córdoba CF,clb_DC United,clb_DSC Arminia Bielefeld,clb_Daegu FC,clb_Dalian YiFang FC,clb_Dalkurd FF,clb_De Graafschap,clb_Defensa y Justicia,clb_Deportes Iquique,clb_Deportes Tolima,clb_Deportivo Alavés,clb_Deportivo Cali,clb_Deportivo Pasto,clb_Deportivo Toluca,clb_Deportivo de La Coruña,clb_Derby 

## Normalització de dades 
Per realitzar una predicció sobre dades numèriques, es donen millors resultats si els valors estan normalitzats ja que el factor multiplicactiu en l'algoritme de machine learning no serà tan irregular. Per tal de normalitzar les dades numèriques s'usarà la puntuació estandar ja que els paràmetres de població són coneguts.

La fórmula per realitzar aquesta predicció és: $Z = \frac{X - \mu}{\sigma}$	essent $X$ el valor observat, $\mu$ la mitjana i $\sigma$ la desviació típica.

Per calcular la mitjana es farà usant ``.mean()``, i per calcular la desviació típica amb ``.std()``

Per veure les columnes que es poden normalitzar usarem ``.describe()`` per saber quins valors numèrics queden:

In [31]:
df.describe()

Age       Overall     Potential         Value           Wage  \
count  17918.000000  17918.000000  17918.000000  1.791800e+04   17918.000000   
mean      25.105257     66.236801     71.329334  2.448629e+06    9883.748186   
std        4.675372      6.929243      6.144098  5.631804e+06   22142.768987   
min       16.000000     46.000000     48.000000  0.000000e+00    1000.000000   
25%       21.000000     62.000000     67.000000  3.250000e+05    1000.000000   
50%       25.000000     66.000000     71.000000  7.000000e+05    3000.000000   
75%       28.000000     71.000000     75.000000  2.100000e+06    9000.000000   
max       45.000000     94.000000     95.000000  1.185000e+08  565000.000000   

       International Reputation     Weak Foot   Skill Moves  \
count              17918.000000  17918.000000  17918.000000   
mean                   1.113908      2.947260      2.362875   
std                    0.395495      0.660106      0.756309   
min                    1.000000      1.000000      1.000000   
25%                    1.000000      3.000000      2.000000   
50%                    1.000000      3.000000      2.000000   
75%                    1.000000      3.000000      3.000000   
max                    5.000000      5.000000      5.000000   

       Contract Valid Until        Height        Weight            LS  \
count          17918.000000  17918.000000  17918.000000  17918.000000   
mean            2020.146668    181.257025    165.968858     57.821926   
std                1.294468      6.730340     15.604084      8.585605   
min             2018.000000    154.940000    110.000000     31.000000   
25%             2019.000000    175.260000    154.000000     53.000000   
50%             2020.000000    180.340000    165.000000     57.821926   
75%             2021.000000    185.420000    176.000000     64.000000   
max             2026.000000    205.740000    243.000000     91.000000   

                 ST            RS            LW            LF            CF  \
count  17918.000000  17918.000000  17918.000000  17918.000000  17918.000000   
mean      57.821926     57.821926     59.046088     58.729750     58.729750   
std        8.585605      8.585605      9.293032      9.253528      9.253528   
min       31.000000     31.000000     25.000000     27.000000     27.000000   
25%       53.000000     53.000000     54.000000     54.000000     54.000000   
50%       57.821926     57.821926     59.046088     58.729750     58.729750   
75%       64.000000     64.000000     65.000000     65.000000     65.000000   
max       91.000000     91.000000     92.000000     93.000000     93.000000   

                 RF            RW           LAM           CAM           RAM  \
count  17918.000000  17918.000000  17918.000000  17918.000000  17918.000000   
mean      58.729750     59.046088     58.994474     58.994474     58.994474   
std        9.253528      9.293032      9.186597      9.186597      9.186597   
min       27.000000     25.000000     27.000000     27.000000     27.000000   
25%       54.000000     54.000000     54.000000     54.000000     54.000000   
50%       58.729750     59.046088     58.994474     58.994474     58.994474   
75%       65.000000     65.000000     65.000000     65.000000     65.000000   
max       93.000000     92.000000     93.000000     93.000000     93.000000   

                 LM           LCM            CM           RCM            RM  \
count  17918.000000  17918.000000  17918.000000  17918.000000  17918.000000   
mean      59.739734     58.223848     58.223848     58.223848     59.739734   
std        8.703003      8.281665      8.281665      8.281665      8.703003   
min       27.000000     30.000000     30.000000     30.000000     27.000000   
25%       55.000000     53.000000     53.000000     53.000000     55.000000   
50%       59.739734     58.223848     58.223848     58.223848     59.739734   
75%       65.000000     63.000000     63.000000     63.000000     65.000000   
max       91.

Les columnes: *Overall, Potential, Height, Weight, LS, ST, RS, LW, LF, CF, RF, RW, LAM, CAM, RAM, LM, LCM, CM, RCM, RM, LWB, LDM, CDM, RDM, RWB, LB, LCB, CB, RCB, RB, Crossin, Finishing, HeadingAccuracy, ShortPassing, Volleys, Dribbling, Curve, FKAccuracy, LongPassing, ,BallControl, Acceleration, SprintSpeed, Agility, Reactions, Balance, ShotPower, Jumping, Stamina, Strength, LongShots, Aggression, Interceptions, Positioning, Vision, Penalties, Composure, Marking, StandingTackle, SlidingTackle, GKDiving, GKHandling, GKKicking, GKPositioning i GKReflexe* on excepte *Height i Weight* estan entre 0 i 100, i *Height i Weight* en valor absolut.

In [32]:
df["Overall"] =(df["Overall"]-df["Overall"].mean())/df["Overall"].std()
df["Potential"] =(df["Potential"]-df["Potential"].mean())/df["Potential"].std()
df["Height"] =(df["Height"]-df["Height"].mean())/df["Height"].std()
df["Weight"] =(df["Weight"]-df["Weight"].mean())/df["Weight"].std()
df["LS"] =(df["LS"]-df["LS"].mean())/df["LS"].std()
df["ST"] =(df["ST"]-df["ST"].mean())/df["ST"].std()
df["RS"] =(df["RS"]-df["RS"].mean())/df["RS"].std()
df["LW"] =(df["LW"]-df["LW"].mean())/df["LW"].std()
df["LF"] =(df["LF"]-df["LF"].mean())/df["LF"].std()
df["CF"] =(df["CF"]-df["CF"].mean())/df["CF"].std()
df["RF"] =(df["RF"]-df["RF"].mean())/df["RF"].std()
df["RW"] =(df["RW"]-df["RW"].mean())/df["RW"].std()
df["LAM"] =(df["LAM"]-df["LAM"].mean())/df["LAM"].std()
df["CAM"] =(df["CAM"]-df["CAM"].mean())/df["CAM"].std()
df["RAM"] =(df["RAM"]-df["RAM"].mean())/df["RAM"].std()
df["LM"] =(df["LM"]-df["LM"].mean())/df["LM"].std()
df["LCM"] =(df["LCM"]-df["LCM"].mean())/df["LCM"].std()
df["CM"] =(df["CM"]-df["CM"].mean())/df["CM"].std()
df["RCM"] =(df["RCM"]-df["RCM"].mean())/df["RCM"].std()
df["RM"] =(df["RM"]-df["RM"].mean())/df["RM"].std()
df["LWB"] =(df["LWB"]-df["LWB"].mean())/df["LWB"].std()
df["LDM"] =(df["LDM"]-df["LDM"].mean())/df["LDM"].std()
df["CDM"] =(df["CDM"]-df["CDM"].mean())/df["CDM"].std()
df["RDM"] =(df["RDM"]-df["RDM"].mean())/df["RDM"].std()
df["RWB"] =(df["RWB"]-df["RWB"].mean())/df["RWB"].std()
df["LB"] =(df["LB"]-df["LB"].mean())/df["LB"].std()
df["LCB"] =(df["LCB"]-df["LCB"].mean())/df["LCB"].std()
df["CB"] =(df["CB"]-df["CB"].mean())/df["CB"].std()
df["RCB"] =(df["RCB"]-df["RCB"].mean())/df["RCB"].std()
df["RB"] =(df["RB"]-df["RB"].mean())/df["RB"].std()
df["Crossing"] =(df["Crossing"]-df["Crossing"].mean())/df["Crossing"].std()
df["Finishing"] =(df["Finishing"]-df["Finishing"].mean())/df["Finishing"].std()
df["HeadingAccuracy"] =(df["HeadingAccuracy"]-df["HeadingAccuracy"].mean())/df["HeadingAccuracy"].std()
df["ShortPassing"] =(df["ShortPassing"]-df["ShortPassing"].mean())/df["ShortPassing"].std()
df["Volleys"] =(df["Volleys"]-df["Volleys"].mean())/df["Volleys"].std()
df["Dribbling"] =(df["Dribbling"]-df["Dribbling"].mean())/df["Dribbling"].std()
df["Curve"] =(df["Curve"]-df["Curve"].mean())/df["Curve"].std()
df["FKAccuracy"] =(df["FKAccuracy"]-df["FKAccuracy"].mean())/df["FKAccuracy"].std()
df["LongPassing"] =(df["LongPassing"]-df["LongPassing"].mean())/df["LongPassing"].std()
df["BallControl"] =(df["BallControl"]-df["BallControl"].mean())/df["BallControl"].std()
df["Acceleration"] =(df["Acceleration"]-df["Acceleration"].mean())/df["Acceleration"].std()
df["SprintSpeed"] =(df["SprintSpeed"]-df["SprintSpeed"].mean())/df["SprintSpeed"].std()
df["Agility"] =(df["Agility"]-df["Agility"].mean())/df["Agility"].std()
df["Reactions"] =(df["Reactions"]-df["Reactions"].mean())/df["Reactions"].std()
df["Balance"] =(df["Balance"]-df["Balance"].mean())/df["Balance"].std()
df["ShotPower"] =(df["ShotPower"]-df["ShotPower"].mean())/df["ShotPower"].std()
df["Jumping"] =(df["Jumping"]-df["Jumping"].mean())/df["Jumping"].std()
df["Stamina"] =(df["Stamina"]-df["Stamina"].mean())/df["Stamina"].std()
df["Strength"] =(df["Strength"]-df["Strength"].mean())/df["Strength"].std()
df["LongShots"] =(df["LongShots"]-df["LongShots"].mean())/df["LongShots"].std()
df["Aggression"] =(df["Aggression"]-df["Aggression"].mean())/df["Aggression"].std()
df["Interceptions"] =(df["Interceptions"]-df["Interceptions"].mean())/df["Interceptions"].std()
df["Positioning"] =(df["Positioning"]-df["Positioning"].mean())/df["Positioning"].std()
df["Vision"] =(df["Vision"]-df["Vision"].mean())/df["Vision"].std()
df["Penalties"] =(df["Penalties"]-df["Penalties"].mean())/df["Penalties"].std()
df["Composure"] =(df["Composure"]-df["Composure"].mean())/df["Composure"].std()
df["Marking"] =(df["Marking"]-df["Marking"].mean())/df["Marking"].std()
df["StandingTackle"] =(df["StandingTackle"]-df["StandingTackle"].mean())/df["StandingTackle"].std()
df["SlidingTackle"] =(df["SlidingTackle"]-df["SlidingTackle"].mean())/df["SlidingTackle"].std()
df["GKDiving"] =(df["GKDiving"]-df["GKDiving"].mean())/df["GKDiving"].std()
df["GKHandling"] =(df["GKHandling"]-df["GKHandling"].mean())/df["GKHandling"].std()
df["GKKicking"] =(df["GKKicking"]-df["GKKicking"].mean())/df["GKKicking"].std()
df["GKPositioning"] =(df["GKPositioning"]-df["GKPositioning"].mean())/df["GKPositioning"].std()
df["GKReflexes"] =(df["GKReflexes"]-df["GKReflexes"].mean())/df["GKReflexes"].std()

df.head()

,Age,Overall,Potential,Value,Wage,International Reputation,Weak Foot,Skill Moves,Contract Valid Until,Height,Weight,LS,ST,RS,LW,LF,CF,RF,RW,LAM,CAM,RAM,LM,LCM,CM,RCM,RM,LWB,LDM,CDM,RDM,RWB,LB,LCB,CB,RCB,RB,Crossing,Finishing,HeadingAccuracy,ShortPassing,Volleys,Dribbling,Curve,FKAccuracy,LongPassing,BallControl,Acceleration,SprintSpeed,Agility,Reactions,Balance,ShotPower,Jumping,Stamina,Strength,LongShots,Aggression,Interceptions,Positioning,Vision,Penalties,Composure,Marking,StandingTackle,SlidingTackle,GKDiving,GKHandling,GKKicking,GKPositioning,GKReflexes,Release Clause,clb_ SSV Jahn Regensburg,clb_1. FC Heidenheim 1846,clb_1. FC Kaiserslautern,clb_1. FC Köln,clb_1. FC Magdeburg,clb_1. FC Nürnberg,clb_1. FC Union Berlin,clb_1. FSV Mainz 05,clb_AC Ajaccio,clb_AC Horsens,clb_AD Alcorcón,clb_ADO Den Haag,clb_AEK Athens,clb_AFC Wimbledon,clb_AIK,clb_AJ Auxerre,clb_AS Béziers,clb_AS Monaco,clb_AS Nancy Lorraine,clb_AS Saint-Étienne,clb_AZ Alkmaar,clb_Aalborg BK,clb_Aarhus GF,clb_Aberdeen,clb_Accrington Stanley,clb_Adelaide United,clb_Ajax,clb_Akhisar Belediyespor,clb_Al Ahli,clb_Al Batin,clb_Al Faisaly,clb_Al Fateh,clb_Al Fayha,clb_Al Hazem,clb_Al Hilal,clb_Al Ittihad,clb_Al Nassr,clb_Al Qadisiyah,clb_Al Raed,clb_Al Shabab,clb_Al Taawoun,clb_Al Wehda,clb_Alanyaspor,clb_Albacete BP,clb_Alianza Petrolera,clb_Amiens SC,clb_América FC (Minas Gerais),clb_América de Cali,clb_Angers SCO,clb_Antalyaspor,clb_Argentinos Juniors,clb_Arka Gdynia,clb_Arsenal,clb_Ascoli,clb_Aston Villa,clb_Atalanta,clb_Athletic Club de Bilbao,clb_Atiker Konyaspor,clb_Atlanta United,clb_Atlético Bucaramanga,clb_Atlético Huila,clb_Atlético Madrid,clb_Atlético Mineiro,clb_Atlético Nacional,clb_Atlético Paranaense,clb_Atlético Tucumán,clb_Audax Italiano,clb_BB Erzurumspor,clb_BK Häcken,clb_BSC Young Boys,clb_Bahia,clb_Barnsley,clb_Bayer 04 Leverkusen,clb_Beijing Renhe FC,clb_Beijing Sinobo Guoan FC,clb_Belgrano de Córdoba,clb_Benevento,clb_Beşiktaş JK,clb_Birmingham City,clb_Blackburn Rovers,clb_Blackpool,clb_Boavista FC,clb_Boca Juniors,clb_Bohemian FC,clb_Bologna,clb_Bolton Wanderers,clb_Borussia Dortmund,clb_Borussia Mönchengladbach,clb_Botafogo,clb_Bournemouth,clb_Boyacá Chicó FC,clb_Bradford City,clb_Bray Wanderers,clb_Brentford,clb_Brescia,clb_Brighton & Hove Albion,clb_Brisbane Roar,clb_Bristol City,clb_Bristol Rovers,clb_Brøndby IF,clb_Burnley,clb_Bursaspor,clb_Burton Albion,clb_Bury,clb_CA Osasuna,clb_CD Antofagasta,clb_CD Aves,clb_CD Everton de Viña del Mar,clb_CD Feirense,clb_CD Huachipato,clb_CD Leganés,clb_CD Lugo,clb_CD Nacional,clb_CD Numancia,clb_CD O'Higgins,clb_CD Palestino,clb_CD Tenerife,clb_CD Tondela,clb_CD Universidad de Concepción,clb_CF Rayo Majadahonda,clb_CF Reus Deportiu,clb_Cagliari,clb_Cambridge United,clb_Cardiff City,clb_Carlisle United,clb_Carpi,clb_Ceará Sporting Club,clb_Celtic,clb_Central Coast Mariners,clb_Cerezo Osaka,clb_Chamois Niortais Football Club,clb_Changchun Yatai FC,clb_Chapecoense,clb_Charlton Athletic,clb_Chelsea,clb_Cheltenham Town,clb_Chicago Fire,clb_Chievo Verona,clb_Chongqing Dangdai Lifan FC SWM Team,clb_Cittadella,clb_Clermont Foot 63,clb_Club América,clb_Club Atlas,clb_Club Atlético Aldosivi,clb_Club Atlético Banfield,clb_Club Atlético Colón,clb_Club Atlético Huracán,clb_Club Atlético Lanús,clb_Club Atlético Talleres,clb_Club Atlético Tigre,clb_Club Brugge KV,clb_Club Deportes Temuco,clb_Club León,clb_Club Necaxa,clb_Club Tijuana,clb_Clube Sport Marítimo,clb_Colchester United,clb_Colo-Colo,clb_Colorado Rapids,clb_Columbus Crew SC,clb_Cork City,clb_Cosenza,clb_Coventry City,clb_Cracovia,clb_Crawley Town,clb_Crewe Alexandra,clb_Crotone,clb_Cruz Azul,clb_Cruzeiro,clb_Crystal Palace,clb_Curicó Unido,clb_Cádiz CF,clb_Córdoba CF,clb_DC United,clb_DSC Arminia Bielefeld,clb_Daegu FC,clb_Dalian YiFang FC,clb_Dalkurd FF,clb_De Graafschap,clb_Defensa y Justicia,clb_Deportes Iquique,clb_Deportes Tolima,clb_Deportivo Alavés,clb_Deportivo Cali,clb_Deportivo Pasto,clb_Deportivo Toluca,clb_Deportivo de La Coruña,clb_Derby 

La resta de , és a dir: _Age, Value, Wage, International Reputation, Weak Foot, Skill Moves, Contract Valid Until i Release Clause_, no s'han normalitzat ja que després de fer un seguit de proves amb aquestes causaven error en el resultat final, per tant no s'ha obtat per fer la normalització d'aquestes

## Predicció

Una vegada s'ha transformat el dataset a un format apte per treballar, procedirem a realitzar la predicció. Per això usarem la llibreria sklearn, usant un sub-conjunt aleatori del dataset, establint una seed perque sempre esculli els mateixos registres.

S'usarà ``.train_test_split()``, passant per paràmetres el dataset, el valor a fer la predicció en aquest cas "*Value*", la proporció del dataset que s'usarà per fer l'entrenament, i un estat per mesclar de la mateixa manera el dataset i sempre el mateix output si es fessin diferentes cridades a aquesta funció.

Aquest retornarà el dataset d'entranament i el dataset de test.

In [33]:
val = df.pop("Value")
X_train, X_test, y_train, y_test = train_test_split(df, val, test_size=0.33, random_state=42)

Miram la longitud del dataset d'entranament:

In [34]:
len(X_train)

12005

Es crea el model de predicció, aquesta serà una regresiò lineal, passant com a paràmentres el dataset d'entranament. Retornarà una instància d'ell mateix.

In [35]:
reg = linear_model.LinearRegression().fit(X_train, y_train)
reg

LinearRegression()

Es realitza la predicció usant un model lineal. Retornarà els valors predits.

In [36]:
preds = reg.predict(X_test)

In [37]:
preds[0]

477666.96018335223

In [38]:
y_test[0]

110500000.0

Per calcular el grau d'acert de l'algorisme, es mirarà el resultat de R-quadrat, mesura estadística de la proximitat de les dades amb la línia de regressió calculada.

In [39]:
r2_score(preds, y_test)

0.967078143938761

Veim que obtenim un R-quadrat del 0.96707, és a dir, el 96.707% dels valors predits es troben dins la recte de regressió lineal. Per tant donat un resultat acceptable, acaba aquesta tercera pràctica de l'assignatura 21722 - Intel·ligència Artificial.